# Table of Contents
* [Preamble](#Preamble)
* [Load TNPS data by MSISDN](#Load-TNPS-data-by-MSISDN)
* [Load TNPS data by NIF](#Load-TNPS-data-by-NIF)
* [Load Call Centre Calls data by MSISDN](#Load-Call-Centre-Calls-data-by-MSISDN)
* [Load Call Centre Calls data by NIF](#Load-Call-Centre-Calls-data-by-NIF)
* [TNPS Oracle stack](#TNPS-Oracle-stack)
    * [Oracle Prepaid segment](#Oracle-Prepaid-segment)
        * [Oracle Prepaid segment - Data Preparation](#Oracle-Prepaid-segment---Data-Preparation)
        * [Oracle Prepaid segment - Training](#Oracle-Prepaid-segment---Training)
        * [Oracle Prepaid segment - Predictions](#Oracle-Prepaid-segment---Predictions)
    * [Oracle Non-Prepaid segments](#Oracle-Non-Prepaid-segments)
        * [Load Oracle Postpaid](#Load-Oracle-Postpaid)
        * [Oracle Non-Prepaid segment - Data Preparation](#Oracle-Non-Prepaid-segment---Data-Preparation)
        * [Oracle Non-Prepaid segment - Training](#Oracle-Non-Prepaid-segment---Training)
        * [Oracle Non-Prepaid segment - Predictions](#Oracle-Non-Prepaid-segment---Predictions)
    * [Write predictions to HDFS](#Write-predictions-to-HDFS)

# Preamble

In [1]:
'''
# Run this on the shell to start Jupyter Notebook

kinit ${USER}@INTERNAL.VODAFONE.COM -k -t .ssh/${USER}.keytab
klist

export BDA_USER_HOME=/var/SP/data/home/${USER}

. ${BDA_USER_HOME}/scripts/properties/common.sh
cd ${BDA_USER_HOME}

ipython notebook --port=65011 --no-browser --ip=127.0.0.1
'''

'\n# Run this on the shell to start Jupyter Notebook\nkinit ${USER}@INTERNAL.VODAFONE.COM -k -t .ssh/${USER}.keytab\nklist\n\nexport BDA_USER_HOME=/var/SP/data/home/${USER}\n\n. ${BDA_USER_HOME}/scripts/properties/common.sh\ncd ${BDA_USER_HOME}\n\nipython notebook --port=65011 --no-browser --ip=127.0.0.1\n'

In [1]:
from common.src.main.python.utils.hdfs_generic import *
import os

MAX_N_EXECUTORS=15
MIN_N_EXECUTORS=1
N_CORES_EXECUTOR=4
EXECUTOR_IDLE_MAX_TIME=120
EXECUTOR_MEMORY='16g'
DRIVER_MEMORY='16g'
N_CORES_DRIVER=1
MEMORY_OVERHEAD=N_CORES_EXECUTOR*2048
QUEUE="root.datascience.normal"
BDA_CORE_VERSION="1.0.0"

# The default directory is chosen from a platform-dependent list, but the user of the 
# application can control the directory location by setting the TMPDIR, TEMP or TMP 
# environment variables.
USER=os.environ.get('USER', '')
os.environ['TMPDIR'] = '/var/SP/data/home/'+USER+'/tmp/'
#os.environ['TEMP'] = '/var/SP/data/home/'+USER+'/tmp/'
#os.environ['TMP'] = '/var/SP/data/home/'+USER+'/tmp/'
import tempfile
print 'Default Temp dir:', tempfile.gettempdir() # prints the current temporary directory

SPARK_COMMON_OPTS=os.environ.get('SPARK_COMMON_OPTS', '')
SPARK_COMMON_OPTS+=" --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
SPARK_COMMON_OPTS+=" --conf spark.port.maxRetries=500"
SPARK_COMMON_OPTS+=" --conf spark.shuffle.manager=tungsten-sort"
SPARK_COMMON_OPTS+=" --queue %s" % QUEUE

# Dynamic allocation configuration
SPARK_COMMON_OPTS+=" --conf spark.shuffle.service.enabled=true"
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.enabled=true"
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)

BDA_ENV = os.environ.get('BDA_USER_HOME', '')

# Attach bda-core-ra codebase
SPARK_COMMON_OPTS+=" --files \
{}/scripts/properties/red_agent/nodes.properties,\
{}/scripts/properties/red_agent/nodes-de.properties,\
{}/scripts/properties/red_agent/nodes-es.properties,\
{}/scripts/properties/red_agent/nodes-ie.properties,\
{}/scripts/properties/red_agent/nodes-it.properties,\
{}/scripts/properties/red_agent/nodes-pt.properties,\
{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS

print os.environ.get('SPARK_COMMON_OPTS', '')
print os.environ.get('PYSPARK_SUBMIT_ARGS', '')

sc, sparkSession, sqlContext = run_sc()
print sc.defaultParallelism

Default Temp dir: /var/SP/data/home/bbergua/tmp
 --queue root.datascience.normal  --conf spark.port.maxRetries=50  --conf spark.network.timeout=10000000  --conf spark.executor.heartbeatInterval=60  --conf spark.yarn.executor.memoryOverhead=2G  --conf spark.sql.broadcastTimeout=1200  --master yarn --conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.kryoserializer.buffer.max=1g --py-files /var/SP/data/home/bbergua/artifacts/bda-core-ra-complete-assembly-2.0.0.jar,/var/SP/data/home/bbergua/artifacts/common.zip,/var/SP/data/home/bbergua/artifacts/graphframes.zip,/var/SP/data/home/bbergua/artifacts/scripts.zip,/var/SP/data/home/bbergua/artifacts/xgboost4j-spark-2.1.1-0.7-jar-with-dependencies.jar --files /var/SP/data/home/bbergua/scripts/properties/red_agent/nodes-de.properties,/var/SP/data/home/bbergua/scripts/properties/red_agent/nodes-es.properties,/var/SP/data/home/bbergua/scripts/properties/red_agent/nodes-ie.properties,/var/SP/data/home/bbergua/scripts/prop

In [2]:
spark = (SparkSession.builder
         .appName("VF-ES Oracle TNPS model")
         .master("yarn")
         .config("spark.submit.deployMode", "client")
         .config("spark.ui.showConsoleProgress", "true")
         .enableHiveSupport()
         .getOrCreate()
         )
print 'Spark version:', spark.version

Spark version: 2.1.0.cloudera1


In [100]:
#spark.stop()

In [3]:
import h2o
from h2o.backend.connection import H2OConnectionError
#if h2o.cluster() is not None:
#    print 'Stopping H2O'
#    h2o.cluster().shutdown()
#if h2o.cluster() is not None:
#    print 'Stopping H2O'
#    h2o.cluster().shutdown()
try:
    #h2o.cluster().show_status()
    h2o.cluster().shutdown()
    print 'Stopped running H2O instance'
except H2OConnectionError as e:
    print e
except:
    pass

In [4]:
import h2o
#h2o.init(nthreads = 1024)
#h2o.init(nthreads = -1, min_mem_size = "28G")
#h2o.init(nthreads = -1, min_mem_size = "28G", max_mem_size = "56G")
#h2o.init(nthreads = -1, max_mem_size = "56G")
h2o.init(nthreads = -1, max_mem_size = "14G")
#h2o.cluster().show_status()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.7.0_79"; Java(TM) SE Runtime Environment (build 1.7.0_79-b15); Java HotSpot(TM) 64-Bit Server VM (build 24.79-b02, mixed mode)
  Starting server from /opt/cloudera/parcels/Anaconda-2.5.0/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/SP/data/home/bbergua/tmp/tmpkz3eP2
  JVM stdout: /var/SP/data/home/bbergua/tmp/tmpkz3eP2/h2o_bbergua_started_from_python.out
  JVM stderr: /var/SP/data/home/bbergua/tmp/tmpkz3eP2/h2o_bbergua_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster version:,3.16.0.2
H2O cluster version age:,4 months and 10 days !!!
H2O cluster name:,H2O_from_python_bbergua_t0ami7
H2O cluster total nodes:,1
H2O cluster free memory:,12.44 Gb
H2O cluster total cores:,56
H2O cluster allowed cores:,56
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [ ]:
import h2o
from h2o.backend.connection import H2OConnectionError
try:
    h2o.cluster().show_status()
except H2OConnectionError as e:
    print e

In [5]:
# Configuration
month = '201712'
label_preds = 'TNPS' # This is the label that we want for output
label_model = 'TNPS2DET' # This is the label used for training the model, and making predictions
#label_model = 'TNPS2PRO'
#filter_cond = 'TNPS4 == "PROMOTER" OR TNPS4 == "HARD DETRACTOR"'
filter_cond = None

In [6]:
import re
import subprocess
import sys
import time
# Spark utils
from pyspark.sql.functions import array_contains, bround, col, collect_set, concat, lit, lpad, size, struct, trim, udf, when
from pyspark.sql.types import IntegerType, StringType, StructField, StructType

Define some useful functions

In [7]:
# Replace column names of the type 'fun(colname)' by 'fun_colname'
# Also replace any character not in [a-zA-Z0-9_.] with '_'
def fix_column_names(df):
    names = df.schema.names

    for n in names:
        m = re.search('([^()]*)\(([^()]*)\)', n)
        if m is not None:
            # print m.group(0), '->', m.group(1) + '_' + m.group(2)
            df = df.withColumnRenamed(n, m.group(1) + '_' + m.group(2))

        m = re.sub('[^a-zA-Z0-9_.]', '_', n)
        if n != m:
            df = df.withColumnRenamed(n, m)

    return df

# Add a prefix to column names of a Dataset
def add_prefix_columns(df, prefix):
    print 'Appending', prefix, 'prefix to columns ...'
    for c in df.columns:
        if c not in ['msisdn', 'nif', 'SEGMENTACION', 'year', 'month', 'day', 'partitioned_month']:
            df = df.withColumnRenamed(c, prefix + '_' + c)
    
    return df

In [8]:
def balance_train_data(all_train_pred_df, label='label'):
    if 'PRED' in all_train_pred_df.columns:
        all_train_pred_df = all_train_pred_df.where('PRED = False').drop('PRED')

    # Get counts by label
    counts = all_train_pred_df.groupBy(label).count().orderBy(label).cache()
    # counts.show()
    counts_dict = dict(counts.rdd.map(lambda x: (x[label], x['count'])).collect())
    
    # Calculate the rate to apply to classes
    fractions_dict = {}
    for l in counts_dict.keys():
        fractions_dict[l] = float(min(counts_dict.values()))/counts_dict[l]
    print 'fractions:', fractions_dict
    
    #all_train_pred_df.select(['TNPS01', 'TNPS', 'TNPS2DET', 'TNPS2PRO', 'TNPS4']).show(1)
    #all_train_pred_df = all_train_pred_df.fillna('', subset=['TNPS01', 'TNPS', 'TNPS2DET', 'TNPS2PRO', 'TNPS4'])
    #all_train_pred_df = all_train_pred_df.fillna('')
    #all_train_pred_df = all_train_pred_df.fillna(0)
    #print all_train_pred_df.count()

    # Columns with missing values:
    #   AC_x_fecha_nacimiento, AC_min_llam_ultmes, AC_num_sms_ultmes, AC_ult3meses_total, AC_media_ult3meses, AC_diasdesdeultrecarga, AC_numrecargasult3meses_total, AC_codigo_postal, AC_cod_golden, AC_fecha_beneficio_activo, 
    
    #print all_train_pred_df.select("AC_fecha_ejecucion").filter(col("AC_fecha_ejecucion").isNull()).count()
    #all_train_pred_df.groupby("AC_fecha_ejecucion").count().show()
    #all_train_pred_df = all_train_pred_df.na.drop(how='any')
    #print all_train_pred_df.count()
    #all_train_pred_df.withColumn("AC_fecha_ejecucion", when(col("AC_fecha_ejecucion").isNull(), "__SOME_PLACEHOLDER__").otherwise(col("AC_fecha_ejecucion")))
    #print all_train_pred_df.select("AC_fecha_ejecucion").filter(col("AC_fecha_ejecucion").isNull()).count()
    #all_train_pred_df.groupby("AC_fecha_ejecucion").count().show()
    balanced = all_train_pred_df.sampleBy(label, fractions=fractions_dict, seed=1234)
    #print balanced.count()
    #balanced = balanced.na.drop(how='any')
    #print balanced.count()
    #balanced.groupBy(label).count().orderBy(label).show()
    #balanced.select(['TNPS01', 'TNPS', 'TNPS2DET', 'TNPS2PRO', 'TNPS4']).show(1)
    #balanced.withColumn("AC_fecha_ejecucion", when(col("AC_fecha_ejecucion").isNull(), "__SOME_PLACEHOLDER__").otherwise(col("AC_fecha_ejecucion")))
    #for c in balanced.columns:
    #    print c
    #    print balanced.select(c).filter(col(c).isNull()).count()
        #balanced.select(c).take(10)
    #print balanced.select("AC_fecha_ejecucion").filter(col("AC_fecha_ejecucion").isNull()).count()
    #balanced.show(1)

    return balanced

def prepare_training_data(df, balance=True, filter_cond=None, repartition_by=None):
    print 'Class count before cleaning:'
    df = df.dropna(subset=label_model)
    df_count = df.count()
    df_groupby = df.groupby(label_model).count()
    df_proportions = df_groupby.withColumn('proportion', 100*df_groupby['count']/df_count).cache()
    df_proportions_dict = dict(df_proportions.select(label_model, 'proportion').collect())
    df_proportions.withColumn('proportion', bround(df_proportions['proportion'], 2)).show()
    
    #df.groupby(label_model).count().show()
    #df = df.na.drop(how='any')
    df = df.drop('msisdn', 'nif')
    
    # If required, filter out rows
    if filter_cond is not None:
        print 'Filtering by', filter_cond
        df = df.filter(filter_cond)

    # Repartition/Coalesce
    # This is due to bug SPARK-18528 which is fixed in versions 2.1.1, and 2.2.0 ( https://issues.apache.org/jira/browse/SPARK-18528 )
    if repartition_by is not None:
        df = df.repartition(repartition_by)
    else:
        df = df.coalesce(1)
    
    #print 'Class count before balancing:'
    #df_before_count = df.count()
    #df_before_groupby = df.groupby(label_model).count()
    #df_before_proportions = df_before_groupby.withColumn('proportion', bround(100*df_before_groupby['count']/df_before_count, 2)).cache()
    #df_before_proportions.show()
    ##df_before_proportions_dict = dict(df_before_proportions.select(label_model, 'proportion').collect())
    
    # If required, balance classes
    balanced = df
    if balance:
        balanced = balance_train_data(df, label_model)
        print 'Class count after balancing:'
        balanced_count = balanced.count()
        balanced_groupby = balanced.groupby(label_model).count()
        balanced_groupby.withColumn('proportion', bround(100*balanced_groupby['count']/balanced_count, 2)).show()
    
    balanced.printSchema()
    #balanced.show(1)
    
    return balanced, df_proportions_dict

def generate_pandas_data(df):
    print time.ctime()
    start = time.time()

    print 'Converting to Pandas'
    df_pd = df.toPandas()

    #cols_to_reencode = []
    print 'Re-encoding String variables to UTF-8'
    #for col in cols_to_reencode:
    #    if col in df_pd.columns:
    #        df_pd[col] = df_pd[col].str.encode('utf-8')
    for col in df_pd.columns.values.tolist():
        if df_pd[col].dtype.kind == 'O':
            #print 'Re-encoding to UTF-8', col
            df_pd[col] = df_pd[col].str.encode('utf-8')

    #df_pd.head()
    
    end = time.time()
    print 'Process took:', "{0:.2f}".format((end - start)/60), 'minutes'

    return df_pd

# Load TNPS data by MSISDN

In [33]:
#del tnps_msisdn
if not 'tnps_msisdn' in globals():
    print 'Reading TNPS by MSISDN from HDFS for month =', month
    tnps_msisdn = spark.read.parquet('/tmp/bbergua/tnps/msisdn/')
    if month is not None:
        tnps_msisdn = tnps_msisdn.filter('partitioned_month == "%s"' % month)
    tnps_msisdn = tnps_msisdn.withColumn('TNPS2DET', 
                                         when(tnps_msisdn['TNPS'] == 'DETRACTOR', tnps_msisdn['TNPS'])
                                         .otherwise('NON DETRACTOR'))
    tnps_msisdn = tnps_msisdn.withColumn('TNPS2PRO', 
                                         when(tnps_msisdn['TNPS'] == 'PROMOTER', tnps_msisdn['TNPS'])
                                         .otherwise('NON PROMOTER'))
    tnps_msisdn = tnps_msisdn.select(['msisdn', 'nif', 'partitioned_month', 'SEGMENTACION',
                                      'TNPS01', 'TNPS', 'TNPS2DET', 'TNPS2PRO', 'TNPS4'])
    tnps_msisdn.printSchema()

# Load TNPS data by NIF

In [9]:
#del tnps_id
if not 'tnps_id' in globals():
    print 'Reading TNPS by Id (NIF) from HDFS for month =', month
    tnps_id = spark.read.parquet('/tmp/bbergua/tnps/id/')
    if month is not None:
        tnps_id = tnps_id.filter('partitioned_month == "%s"' % month)
    tnps_id = tnps_id.withColumn('TNPS2DET', 
                                 when(tnps_id['TNPS'] == 'DETRACTOR', tnps_id['TNPS'])
                                 .otherwise('NON DETRACTOR'))
    tnps_id = tnps_id.withColumn('TNPS2PRO', 
                                 when(tnps_id['TNPS'] == 'PROMOTER', tnps_id['TNPS'])
                                 .otherwise('NON PROMOTER'))
    tnps_id = tnps_id.select(['nif', 'partitioned_month', 'SEGMENTACION_Prepaid', 'SEGMENTACION',
                              'TNPS01', 'TNPS', 'TNPS2DET', 'TNPS2PRO', 'TNPS4'])
    tnps_id.printSchema()

Reading TNPS by Id (NIF) from HDFS for month = 201712
root
 |-- nif: string (nullable = true)
 |-- partitioned_month: string (nullable = true)
 |-- SEGMENTACION_Prepaid: string (nullable = true)
 |-- SEGMENTACION: string (nullable = true)
 |-- TNPS01: integer (nullable = true)
 |-- TNPS: string (nullable = true)
 |-- TNPS2DET: string (nullable = true)
 |-- TNPS2PRO: string (nullable = true)
 |-- TNPS4: string (nullable = true)



# Load Call Centre Calls data by MSISDN

In [35]:
#del ccc_msisdn
if not 'ccc_msisdn' in globals():
    print 'Reading CCC by MSISDN from HDFS for month =', month
    ccc_msisdn = spark.read.parquet('/tmp/bbergua/ccc/msisdn/')
    if month is not None:
        ccc_msisdn = ccc_msisdn.filter('partitioned_month == "%s"' % month)
    ccc_msisdn = add_prefix_columns(ccc_msisdn, 'CCC')
    ccc_msisdn.groupby('partitioned_month').count().show()
    ccc_msisdn.printSchema()

# Load Call Centre Calls data by NIF

In [10]:
#del ccc_id
if not 'ccc_id' in globals():
    print 'Reading CCC by Id (NIF) from HDFS for month =', month
    ccc_id = spark.read.parquet('/tmp/bbergua/ccc/id/')
    if month is not None:
        ccc_id = ccc_id.filter('partitioned_month == "%s"' % month)
    ccc_id = add_prefix_columns(ccc_id, 'CCC')
    ccc_id.groupby('partitioned_month').count().show()
    ccc_id.printSchema()

Reading CCC by Id (NIF) from HDFS for month = 201712
Appending CCC prefix to columns ...
+-----------------+-------+
|partitioned_month|  count|
+-----------------+-------+
|           201712|1115576|
+-----------------+-------+

root
 |-- nif: string (nullable = true)
 |-- partitioned_month: string (nullable = true)
 |-- CCC_Pagar_menos: long (nullable = true)
 |-- CCC_Incidencia_Provision_Neba: long (nullable = true)
 |-- CCC_Incidencia_Provision_Fibra: long (nullable = true)
 |-- CCC_Incidencia_Provision_DSL: long (nullable = true)
 |-- CCC_Incidencia_Tecnica: long (nullable = true)
 |-- CCC_Incidencia_SGI: long (nullable = true)
 |-- CCC_Incidencia_Resto: long (nullable = true)
 |-- CCC_Incidencia_Provision_Movil: long (nullable = true)
 |-- CCC_Resultado_No_Aplica: long (nullable = true)
 |-- CCC_Resultado_Informacion: long (nullable = true)
 |-- CCC_Resultado_Solucionado: long (nullable = true)
 |-- CCC_Resultado_Retenido: long (nullable = true)
 |-- CCC_Resultado_No_Retenido: lo

# TNPS Oracle stack

## Oracle Prepaid segment

### Oracle Prepaid segment - Data Preparation

In [37]:
##print 'ACC CAR rows:', vf_pre.filter('partitioned_month == "%s"' % month).count()
#print 'Accenture CAR distinct MSISDN:', spark.table('udf_es.pre_explicativas_4m_' + month).select('msisdn').distinct().count()
#print 'Accenture CAR distinct NIF:   ', spark.table('udf_es.pre_explicativas_4m_' + month).select('nif').distinct().count()
##print 'AC_FINAL_PRE rows:', spark.table('raw_es.vf_pre_ac_final').filter('partitioned_month == "%s"' % month).count()
#print 'AC_FINAL_PREP distinct MSISDN:', spark.table('raw_es.vf_pre_ac_final').filter('partitioned_month == "%s"' % month).select('msisdn').distinct().count()
#print 'AC_FINAL_PREP distinct NIF:   ', spark.table('raw_es.vf_pre_ac_final').filter('partitioned_month == "%s"' % month).select('num_documento_comprador').distinct().count()

In [38]:
def prepare_dataset_prepaid():
    # Load Oracle AC Final Prepaid
    #del vf_pre
    #if not 'vf_pre' in globals():
    #print 'Load Oracle Prepaid CAR from Hive for month =', month
    #vf_pre = spark.table('udf_es.pre_explicativas_4m_' + month)
    print 'Load Oracle AC Final Prepaid from Hive for month =', month
    vf_pre = spark.table('raw_es.vf_pre_ac_final')
    if month is not None:
        vf_pre = vf_pre.filter('partitioned_month == "%s"' % month)
    vf_pre.groupby('partitioned_month').count().show()
    vf_pre = vf_pre.drop('num_documento_cliente', 'sfid_canje')
    vf_pre = vf_pre.withColumnRenamed('num_documento_comprador', 'nif')
    vf_pre = vf_pre.withColumn('tipo_documento_cliente', when(vf_pre['tipo_documento_cliente'] == 'Tarj.Residente',  'Tarj. Residente')
                                                        .when(vf_pre['tipo_documento_cliente'] == 'NIE',             'Tarj. Residente')
                                                        .when(vf_pre['tipo_documento_cliente'] == 'T_RES',           'Tarj. Residente')
                                                        .when(vf_pre['tipo_documento_cliente'] == 'TARJ. RESIDENTE', 'Tarj. Residente')
                                                        .when(vf_pre['tipo_documento_cliente'] == 'PASAPORTE',       'Pasaporte')
                                                        .when(vf_pre['tipo_documento_cliente'] == 'GMT+1',           '')
                                                        .otherwise(vf_pre['tipo_documento_cliente']))
    vf_pre = add_prefix_columns(vf_pre, 'AC')
    #vf_pre = vf_pre.withColumnRenamed('nif', 'AC_nif')
    #vf_pre.printSchema()

    # Join Prepaid data with Call Centre Calls
    print 'Join Prepaid data with Call Centre Calls'
    vf_pre_ccc = vf_pre.join(ccc_msisdn, ['msisdn', 'partitioned_month'], 'left_outer')
    ccc_columns = [x for x in ccc_msisdn.columns if x not in ['msisdn', 'partitioned_month']]
    vf_pre_ccc = vf_pre_ccc.fillna(0, ccc_columns)
    #vf_pre_ccc.groupby('partitioned_month').count().show()
    #vf_pre_ccc.printSchema()

    # Join Prepaid data with TNPS (to get the label)
    #tnps_msisdn.show()
    print 'Join TNPS with Prepaid data'
    tnps_msisdn_prepaid = tnps_msisdn.filter('SEGMENTACION == "Prepaid"').drop('SEGMENTACION').select(['msisdn', 'partitioned_month', 'TNPS01', 'TNPS', 'TNPS2DET', 'TNPS2PRO', 'TNPS4'])
    #tnps_msisdn_prepaid.groupby('partitioned_month').count().show()
    tnps_msisdn_prepaid.groupby('TNPS4').count().show()
    vf_pre_ccc_tnps = vf_pre_ccc.join(tnps_msisdn_prepaid, ['msisdn', 'partitioned_month'], 'left_outer')
    #vf_pre_ccc_tnps.groupby('partitioned_month').count().show()
    
    # Now, generate dataset for model training
    vf_pre = vf_pre_ccc_tnps.drop('year', 'month', 'day')
    vf_pre = fix_column_names(vf_pre)

    # Calculate proportions of TNPS4 levels
    vf_pre_count = vf_pre.na.drop(subset='TNPS4').count()
    vf_pre_groupby = vf_pre.na.drop(subset='TNPS4').groupby('TNPS4').count()
    vf_pre_proportions = vf_pre_groupby.withColumn('proportion', 100*vf_pre_groupby['count']/vf_pre_count).cache()
    vf_pre_proportions_dict = dict(vf_pre_proportions.select('TNPS4', 'proportion').collect())
    vf_pre_proportions.withColumn('proportion', bround(vf_pre_proportions['proportion'], 2)).show()
    
    vf_pre.printSchema()
    
    return vf_pre, vf_pre_proportions_dict

In [39]:
vf_pre, vf_pre_proportions = prepare_dataset_prepaid()

Load Oracle AC Final Prepaid from Hive for month = 201712
+-----------------+-------+
|partitioned_month|  count|
+-----------------+-------+
|           201712|2730651|
+-----------------+-------+

Appending AC prefix to columns ...
Join Prepaid data with Call Centre Calls
Join TNPS with Prepaid data
+--------------+-----+
|         TNPS4|count|
+--------------+-----+
|HARD DETRACTOR| 6972|
|       NEUTRAL| 7111|
|      PROMOTER|30105|
|SOFT DETRACTOR| 4827|
+--------------+-----+

+--------------+-----+----------+
|         TNPS4|count|proportion|
+--------------+-----+----------+
|HARD DETRACTOR| 6972|     14.22|
|       NEUTRAL| 7111|     14.51|
|      PROMOTER|30105|     61.42|
|SOFT DETRACTOR| 4827|      9.85|
+--------------+-----+----------+

root
 |-- msisdn: string (nullable = true)
 |-- partitioned_month: string (nullable = true)
 |-- AC_fecha_ejecucion: string (nullable = true)
 |-- nif: string (nullable = true)
 |-- AC_fx_1llamada: string (nullable = true)
 |-- AC_fx_activ

In [40]:
#spark.table('raw_es.vf_pre_ac_final').filter('partitioned_month == "%s"' % month).groupby('tipo_documento_cliente').count().sort('count', ascending=False).show()
#vf_pre.groupby('AC_tipo_documento_cliente').count().sort('count', ascending=False).show()

In [41]:
vf_pre_proportions

{u'HARD DETRACTOR': 14.224217076405182,
 u'NEUTRAL': 14.507803733550954,
 u'PROMOTER': 61.419973477506886,
 u'SOFT DETRACTOR': 9.84800571253698}

### Oracle Prepaid segment - Training

In [42]:
vf_pre_training, vf_pre_training_proportions = prepare_training_data(vf_pre)

Class count before cleaning:
+-------------+-----+----------+
|     TNPS2DET|count|proportion|
+-------------+-----+----------+
|NON DETRACTOR|37216|     75.93|
|    DETRACTOR|11799|     24.07|
+-------------+-----+----------+

fractions: {u'NON DETRACTOR': 0.31704105760963025, u'DETRACTOR': 1.0}
Class count after balancing:
+-------------+-----+----------+
|     TNPS2DET|count|proportion|
+-------------+-----+----------+
|    DETRACTOR|11799|     50.17|
|NON DETRACTOR|11720|     49.83|
+-------------+-----+----------+

root
 |-- partitioned_month: string (nullable = true)
 |-- AC_fecha_ejecucion: string (nullable = true)
 |-- AC_fx_1llamada: string (nullable = true)
 |-- AC_fx_activacion: string (nullable = true)
 |-- AC_codigo_plan_precios: string (nullable = true)
 |-- AC_fx_ultima_recarga: string (nullable = true)
 |-- AC_estado_servicio: string (nullable = true)
 |-- AC_tipo_documento_cliente: string (nullable = true)
 |-- AC_tipo_documento_comprador: string (nullable = true)
 |--

We need to properly encode columns that contain special characters. For example, columns *AC_nacionalidad* (AC_FINAL_PREPAID) or *CAR_seg_nacionalidad* (Accenture CAR) contains the 'ñ' character. Other columns also contain special characters, for example, *AC_num_documento_cliente*, and *AC_nif* (AC_FINAL_PREPAID).

In [44]:
print time.ctime()
start = time.time()
print vf_pre_training.count()
vf_pre_training.write.mode('overwrite').format('parquet').save('/tmp/bbergua/tmp/vf_pre_training-'+month)
end = time.time()
print 'Process took:', "{0:.2f}".format((end - start)/60), 'minutes'

Mon Apr  9 09:45:54 2018
23519
Process took: 2.62 minutes


In [45]:
# $ hdfs getconf -confKey fs.defaultFS
# hdfs://nameservice1
# $ hdfs getconf -namenodes
# vgddp350hr.dc.sedc.internal.vodafone.com vgddp351hr.dc.sedc.internal.vodafone.com
out = subprocess.check_output('hdfs getconf -namenodes', shell=True)
nodes = ['hdfs://'+n for n in out.strip().split(' ')]

In [46]:
path_vf_pre_training = [n+'/tmp/bbergua/tmp/vf_pre_training-'+month for n in nodes]
path_vf_pre_training[0]

'hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/tmp/bbergua/tmp/vf_pre_training-201712'

In [47]:
i=0
from h2o.backend.connection import H2OServerError
while i < len(nodes):
    try:
        vf_pre_training_df_h2o = h2o.import_file(path=path_vf_pre_training[i])
        vf_pre_training_df_h2o.nrows
        break
    except H2OServerError as e:
        i=i+1
        #print e
#subprocess.call('hdfs dfs -rm -r /tmp/bbergua/tmp/vf_pre_training-'+month, shell=True)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [48]:
if False:
    vf_pre_training_pd = generate_pandas_data(vf_pre_training)
    # Copy to H2O

    h2o.remove_all()
    #print balanced_pd.columns.values.tolist()
    #os.environ["PYTHONIOENCODING"] = "UTF-8"
    print 'Copying data to H2O'
    vf_pre_training_df_h2o = h2o.H2OFrame(vf_pre_training_pd, destination_frame='data_df_h2o.hex', header=1)
    h2o.ls()
    vf_pre_training_df_h2o.head()

In [90]:
vf_pre_training_df_h2o['AC_fx_1llamada'] = vf_pre_training_df_h2o['AC_fx_1llamada'].asnumeric()
vf_pre_training_df_h2o['AC_fx_activacion'] = vf_pre_training_df_h2o['AC_fx_activacion'].asnumeric()
vf_pre_training_df_h2o['AC_fx_ultima_recarga'] = vf_pre_training_df_h2o['AC_fx_ultima_recarga'].asnumeric()
vf_pre_training_df_h2o['AC_x_fecha_nacimiento'] = vf_pre_training_df_h2o['AC_x_fecha_nacimiento'].asnumeric()
vf_pre_training_df_h2o['AC_fecha_beneficio_activo'] = vf_pre_training_df_h2o['AC_fecha_beneficio_activo'].asnumeric()

vf_pre_training_df_h2o['AC_num_prepago'] = vf_pre_training_df_h2o['AC_num_prepago'].asnumeric()
vf_pre_training_df_h2o['AC_num_pospago'] = vf_pre_training_df_h2o['AC_num_pospago'].asnumeric()
vf_pre_training_df_h2o['AC_num_total'] = vf_pre_training_df_h2o['AC_num_total'].asnumeric()
vf_pre_training_df_h2o['AC_min_llam_ultmes'] = vf_pre_training_df_h2o['AC_min_llam_ultmes'].asnumeric()
vf_pre_training_df_h2o['AC_num_sms_ultmes'] = vf_pre_training_df_h2o['AC_num_sms_ultmes'].asnumeric()
vf_pre_training_df_h2o['AC_ult3meses_total'] = vf_pre_training_df_h2o['AC_ult3meses_total'].asnumeric()
vf_pre_training_df_h2o['AC_media_ult3meses'] = vf_pre_training_df_h2o['AC_media_ult3meses'].asnumeric()
vf_pre_training_df_h2o['AC_diasdesdeultrecarga'] = vf_pre_training_df_h2o['AC_diasdesdeultrecarga'].asnumeric()
vf_pre_training_df_h2o['AC_numrecargasult3meses_total'] = vf_pre_training_df_h2o['AC_numrecargasult3meses_total'].asnumeric()

#vf_pre_training_df_h2o[''] = vf_pre_training_df_h2o[''].asnumeric()

In [91]:
vf_pre_training_df_h2o.describe()

Rows:23519
Cols:101




,partitioned_month,AC_fecha_ejecucion,AC_fx_1llamada,AC_fx_activacion,AC_codigo_plan_precios,AC_fx_ultima_recarga,AC_estado_servicio,AC_tipo_documento_cliente,AC_tipo_documento_comprador,AC_nacionalidad,AC_x_fecha_nacimiento,AC_tac_fac,AC_num_prepago,AC_num_pospago,AC_num_total,AC_min_llam_ultmes,AC_num_sms_ultmes,AC_ult3meses_total,AC_media_ult3meses,AC_diasdesdeultrecarga,AC_numrecargasult3meses_total,AC_codigo_postal,AC_cod_golden,AC_cobertura_4g,AC_lortad,AC_deuda,AC_flag_huella_ono,AC_flag_4g_aperturas,AC_flag_4g_nodos,AC_flag_huella_vf,AC_flag_huella_neba,AC_flag_huella_euskaltel,AC_flag_cobertura_adsl,AC_flag_beneficio_activo,AC_fecha_beneficio_activo,CCC_Pagar_menos,CCC_Incidencia_Provision_Neba,CCC_Incidencia_Provision_Fibra,CCC_Incidencia_Provision_DSL,CCC_Incidencia_Tecnica,CCC_Incidencia_SGI,CCC_Incidencia_Resto,CCC_Incidencia_Provision_Movil,CCC_Resultado_No_Aplica,CCC_Resultado_Informacion,CCC_Resultado_Solucionado,CCC_Resultado_Retenido,CCC_Resultado_No_Retenido,CCC_Resultado_Escalo,CCC_Resultado_Envio_tecnico,CCC_Resultado_Transferencia,CCC_Resultado_Abono,CCC_Resultado_Bajas,CCC_Resultado_Reclamacion,CCC_Desactivacion_BA_Movil_TV,CCC_Desactivacion_TV,CCC_Desactivacion_Movil,CCC_Desactivacion_Total,CCC_Desactivacion_NET,CCC_Desactivacion_Fijo,CCC_Desactivacion_USB,CCC_Desactivacion_Resto,CCC_Ofrecimiento,CCC_Transferencia,CCC_Cobro,CCC_Precios,CCC_Portabilidad_Inversa,CCC_Portabilidad,CCC_Informacion,CCC_Cierre,CCC_Alta,CCC_Factura,CCC_Averia_DSL,CCC_Averia_Fibra,CCC_Averia_TV,CCC_Averia_Resto,CCC_Averia_Neba,CCC_Averia_Modem_Router,CCC_Baja,CCC_Consulta_Tecnica_TV,CCC_Consulta_Tecnica_Fibra,CCC_Consulta_Tecnica_Neba,CCC_Consulta_Tecnica_DSL,CCC_Consulta_Tecnica_Movil,CCC_Consulta_Tecnica_Modem_Router,CCC_Consulta_Tecnica_Resto,CCC_Consulta_Ficha,CCC_Consulta_Resto,CCC_Productos_Voz,CCC_Productos_Datos,CCC_Productos_Resto,CCC_Provision_Neba,CCC_Provision_Fibra,CCC_Provision_DSL,CCC_Provision_Resto,CCC_Provision_Movil,TNPS01,TNPS,TNPS2DET,TNPS2PRO,TNPS4
type,enum,enum,int,int,enum,int,enum,enum,enum,enum,int,enum,int,int,int,int,int,int,int,int,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,enum,enum,enum,enum
mins,,,17530101.0,19700101.0,,17530101.0,,,,,0.0,,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,,,,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,
mean,,,20126383.582,20139657.2485,,19985446.9337,,,,,26.8795441983,,1.48735065266,0.196054254007,1.68340490667,2849.36153748,17.6866363366,397.847102343,313.502487351,87.5212381479,19.3187635529,,,,,,,,,,,,,,8.48479952379,0.0,0.000170075258302,8.50376291509e-05,0.000170075258302,0.000170075258302,0.0,8.50376291509e-05,0.0,0.00880139461712,0.226922913389,0.137038139377,0.00667545388835,0.00403928738467,0.00348654279519,0.000552744589481,0.00943917683575,0.000212594072877,0.0,0.00229601598707,0.000680301033207,0.000212594072877,0.00939665802117,0.0,0.00212594072877,0.000637782218632,0.000212594072877,0.000340150516604,0.00136060206641,0.137803478039,0.00608019048429,0.0,4.25188145754e-05,0.0342276457332,0.123559675156,0.0167098941282,0.0307411029381,0.0114800799354,0.00255112887453,0.00204090309962,0.000552744589481,0.00233853480165,0.000765338662358,8.50376291509e-05,0.00361409923891,4.25188145754e-05,0.00046770696033,0.000127556443726,0.000722819847783,0.0273821165866,0.000722819847783,0.000297631702028,0.0,0.0,4.25188145754e-05,0.000255112887453,0.240443896424,0.000170075258302,0.000297631702028,0.000722819847783,0.000340150516604,4.25188145754e-05,5.92559207449,,,,
maxs,,,20171231.0,2017

In [92]:
#data_df_h2o.types

In [93]:
# Split the data into Train/Test/Validation with Train having 70% and test and validation 15% each
train,test,valid = vf_pre_training_df_h2o.split_frame(ratios=[.7, .15])

In [94]:
# Identify predictors and response
#x = train.columns
y = label_model
#x.remove(y)
x = [item for item in train.columns if item not in 
     ['msisdn', 'nif', 'TNPS01', 'TNPS2DET', 'TNPS2PRO', 'TNPS', 'TNPS4',
      'AC_cod_golden', 'AC_tac_fac']]

In [95]:
train.types[y]

u'enum'

In [96]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
valid[y] = valid[y].asfactor()

In [97]:
train.types[y]

u'enum'

In [98]:
from h2o.automl import H2OAutoML
# Run AutoML for 60 seconds
aml = H2OAutoML(max_runtime_secs = 60, seed=1234)
aml.train(x = x, y = y,
          training_frame = train,
          leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [99]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb

model_id,auc,logloss
StackedEnsemble_AllModels_0_AutoML_20180409_103218,0.548555,0.689099
StackedEnsemble_BestOfFamily_0_AutoML_20180409_103218,0.548505,0.689104
GLM_grid_0_AutoML_20180409_103218_model_0,0.54594,0.690217
GBM_grid_0_AutoML_20180409_103218_model_0,0.539046,0.692264
DRF_0_AutoML_20180409_103218,0.531176,0.706489
XRT_0_AutoML_20180409_103218,0.529478,0.70633


In [100]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_0_AutoML_20180409_103218
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.215825318863
RMSE: 0.464570036553
LogLoss: 0.624151737299
Null degrees of freedom: 13104
Residual degrees of freedom: 13100
Null deviance: 18167.1396811
Residual deviance: 16359.0170346
AIC: 16369.0170346
AUC: 0.916394002913
Gini: 0.832788005826
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.500244202923: 


,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,5260.0,1321.0,0.2007,(1321.0/6581.0)
NON DETRACTOR,930.0,5594.0,0.1426,(930.0/6524.0)
Total,6190.0,6915.0,0.1718,(2251.0/13105.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.5002442,0.8325024,191.0
max f2,0.4733915,0.8911676,238.0
max f0point5,0.5193943,0.8554511,157.0
max accuracy,0.5060606,0.8305990,181.0
max precision,0.7484017,1.0,0.0
max recall,0.2697283,1.0,384.0
max specificity,0.7484017,1.0,0.0
max absolute_mcc,0.5117139,0.6614098,172.0
max min_per_class_accuracy,0.5044461,0.8284795,184.0
max mean_per_class_accuracy,0.5044461,0.8305898,184.0


Gains/Lift Table: Avg response rate: 49.78 %



...

,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,95.0,6486.0,0.9856,(6486.0/6581.0)
NON DETRACTOR,38.0,6486.0,0.0058,(38.0/6524.0)
Total,133.0,12972.0,0.4978,(6524.0/13105.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3815602,0.6653673,349.0
max f2,0.1529387,0.8321216,399.0
max f0point5,0.4375481,0.5576618,296.0
max accuracy,0.5034567,0.5337657,168.0
max precision,0.6610970,0.6666667,4.0
max recall,0.1529387,1.0,399.0
max specificity,0.8044995,0.9998480,0.0
max absolute_mcc,0.4885030,0.0679144,200.0
max min_per_class_accuracy,0.5010706,0.5323421,173.0
max mean_per_class_accuracy,0.5034567,0.5336613,168.0


Gains/Lift Table: Avg response rate: 49.78 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100725,0.5893328,1.2630695,1.2630695,0.6287879,0.6287879,0.0127223,0.0127223,26.3069459,26.3069459
,2,0.0200687,0.5779948,1.1807080,1.2220453,0.5877863,0.6083650,0.0118026,0.0245248,18.0707991,22.2045306
,3,0.0300649,0.5700745,1.0733709,1.1726130,0.5343511,0.5837563,0.0107296,0.0352544,7.3370901,17.2612953
,4,0.0400610,0.5652431,1.1500403,1.1669805,0.5725191,0.5809524,0.0114960,0.0467505,15.0040251,16.6980526
,5,0.0500572,0.5614923,1.1040386,1.1544113,0.5496183,0.5746951,0.0110362,0.0577866,10.4038641,15.4411339
,6,0.1000382,0.5475180,1.1071054,1.1307764,0.5511450,0.5629291,0.0553342,0.1131208,10.7105415,13.0776419
,7,0.1500191,0.5378468,1.0611038,1.1075640,0.5282443,0.5513733,0.0530349,0.1661557,6.1103805,10.7564027
,8,0.2,0.5306248,1.0549703,1.0944206,0.5251908,0.5448302,0.0527284,0.2188841,5.4970257,9.4420601
,9,0.3000382,0.5198274,1.0602944,1.0830423,0.5278413,0.5391658,0.1060699,0.3249540,6.0294420,8.3042314
,10,0.4,0.5100037,1.0595704,1.0771766,0.5274809,0.5362457,0.1059166,0.4308706,5.9570418,7.7176579


Model metrics of train dataset for max accuracy (if BinomialModel):

In [101]:
from h2o.model.metrics_base import H2OBinomialModelMetrics#, H2OMultinomialModelMetrics
train_metrics = aml.leader.model_performance()
if isinstance(train_metrics, H2OBinomialModelMetrics):
    #print aml.leader.confusion_matrix(metrics="accuracy")
    print train_metrics.confusion_matrix(metrics="accuracy")
    print 'AUC:', train_metrics.auc()
    print 'Accuracy:', train_metrics.accuracy()
    #train_metrics.plot(type='roc', server=True)

Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.50606064703: 


,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,5561.0,1020.0,0.155,(1020.0/6581.0)
NON DETRACTOR,1200.0,5324.0,0.1839,(1200.0/6524.0)
Total,6761.0,6344.0,0.1694,(2220.0/13105.0)



AUC: 0.916394002913
Accuracy: [[0.5060606470296386, 0.8305990080122091]]


In [102]:
#train_metrics.plot(type='roc', server=True)

Model metrics of test dataset, and confusion matrix for max accuracy (if BinomialModel):

In [103]:
#test_pred = aml.leader.predict(test)
test_metrics = aml.leader.model_performance(test_data=test)
print test_metrics


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.247973432803
RMSE: 0.497969309098
LogLoss: 0.689098530378
Null degrees of freedom: 3586
Residual degrees of freedom: 3582
Null deviance: 4972.40126705
Residual deviance: 4943.59285693
AIC: 4953.59285693
AUC: 0.548555405903
Gini: 0.0971108118057
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.415292952739: 


,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,80.0,1731.0,0.9558,(1731.0/1811.0)
NON DETRACTOR,30.0,1746.0,0.0169,(30.0/1776.0)
Total,110.0,3477.0,0.4909,(1761.0/3587.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4152930,0.6647630,349.0
max f2,0.2639327,0.8307606,397.0
max f0point5,0.4491535,0.5586344,305.0
max accuracy,0.4974340,0.5372177,198.0
max precision,0.5878169,0.6956522,21.0
max recall,0.2639327,1.0,397.0
max specificity,0.8105789,0.9994478,0.0
max absolute_mcc,0.4491535,0.0793813,305.0
max min_per_class_accuracy,0.5056392,0.5343468,178.0
max mean_per_class_accuracy,0.4916537,0.5384196,214.0


Gains/Lift Table: Avg response rate: 49.51 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100362,0.5899129,1.3464715,1.3464715,0.6666667,0.6666667,0.0135135,0.0135135,34.6471471,34.6471471
,2,0.0200725,0.5805125,1.0659566,1.2062140,0.5277778,0.5972222,0.0106982,0.0242117,6.5956582,20.6214027
,3,0.0301087,0.5745599,1.0098536,1.1407606,0.5,0.5648148,0.0101351,0.0343468,0.9853604,14.0760552
,4,0.0401450,0.5683727,0.8976476,1.0799823,0.4444444,0.5347222,0.0090090,0.0433559,-10.2352352,7.9982326
,5,0.0501812,0.5650378,1.0659566,1.0771772,0.5277778,0.5333333,0.0106982,0.0540541,6.5956582,7.7177177
,6,0.1000836,0.5499833,1.1508946,1.1139332,0.5698324,0.5515320,0.0574324,0.1114865,15.0894610,11.3933223
,7,0.1499861,0.5421807,1.0154952,1.0811816,0.5027933,0.5353160,0.0506757,0.1621622,1.5495244,8.1181553
,8,0.2001673,0.5355407,1.0771772,1.0801777,0.5333333,0.5348189,0.0540541,0.2162162,7.7177177,8.0177671
,9,0.2999721,0.5243192,1.1283280,1.0961980,0.5586592,0.5427509,0.1126126,0.3288288,12.8328049,9.6197964
,10,0.4000558,0.5149910,1.0182925,1.0767080,0.5041783,0.5331010,0.1019144,0.4307432,1.8292492,7.6708023


In [104]:
#print test_metrics.__class__
#if str(test_metrics.__class__) == "<class 'h2o.model.metrics_base.H2OBinomialModelMetrics'>":
from h2o.model.metrics_base import H2OBinomialModelMetrics
if isinstance(test_metrics, H2OBinomialModelMetrics):
    print test_metrics.confusion_matrix(metrics="accuracy")
    print 'AUC:', test_metrics.auc()
    print 'Accuracy:', test_metrics.accuracy()

Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.497434049034: 


,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,837.0,974.0,0.5378,(974.0/1811.0)
NON DETRACTOR,686.0,1090.0,0.3863,(686.0/1776.0)
Total,1523.0,2064.0,0.4628,(1660.0/3587.0)



AUC: 0.548555405903
Accuracy: [[0.4974340490344474, 0.5372177306941734]]


In [105]:
print 'Accuracy:', test_metrics.accuracy()
print 'Threshold max acc:', test_metrics.find_threshold_by_max_metric('accuracy')
print 'Idx max acc:', test_metrics.find_idx_by_threshold(test_metrics.find_threshold_by_max_metric('accuracy'))
#test_metrics.plot(type='roc', server=True)

Accuracy: [[0.4974340490344474, 0.5372177306941734]]
Threshold max acc: 0.497434049034
Idx max acc: 198


### Oracle Prepaid segment - Predictions

Make predictions

In [106]:
vf_pre.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- partitioned_month: string (nullable = true)
 |-- AC_fecha_ejecucion: string (nullable = true)
 |-- nif: string (nullable = true)
 |-- AC_fx_1llamada: string (nullable = true)
 |-- AC_fx_activacion: string (nullable = true)
 |-- AC_codigo_plan_precios: string (nullable = true)
 |-- AC_fx_ultima_recarga: string (nullable = true)
 |-- AC_estado_servicio: string (nullable = true)
 |-- AC_tipo_documento_cliente: string (nullable = true)
 |-- AC_tipo_documento_comprador: string (nullable = true)
 |-- AC_nacionalidad: string (nullable = true)
 |-- AC_x_fecha_nacimiento: string (nullable = true)
 |-- AC_tac_fac: string (nullable = true)
 |-- AC_num_prepago: string (nullable = true)
 |-- AC_num_pospago: string (nullable = true)
 |-- AC_num_total: string (nullable = true)
 |-- AC_min_llam_ultmes: string (nullable = true)
 |-- AC_num_sms_ultmes: string (nullable = true)
 |-- AC_ult3meses_total: string (nullable = true)
 |-- AC_media_ult3meses: strin

In [107]:
h2o.ls()

,key
0,AutoML_20180409_095222
1,AutoML_20180409_103218
2,AutoML_Feedback_AutoML_20180409_095222
3,AutoML_Feedback_AutoML_20180409_103218
4,AutoML_Leaderboard_automl_py_48_sid_88f9
5,AutoML_Leaderboard_automl_py_8_sid_88f9
6,DRF_0_AutoML_20180409_095222
7,DRF_0_AutoML_20180409_095222_cv_1
8,DRF_0_AutoML_20180409_095222_cv_2
9,DRF_0_AutoML_20180409_095222_cv_3


In [108]:
if False:
    #h2o.remove_all()
    vf_pre_all_pd = generate_pandas_data(vf_pre)

    # Copy to H2O

    #print balanced_pd.columns.values.tolist()
    #os.environ["PYTHONIOENCODING"] = "UTF-8"
    print 'Copying data to H2O'
    all_data_df_h2o = h2o.H2OFrame(vf_pre_all_pd, destination_frame='all_data_df_h2o.hex', header=1)
    all_data_df_h2o.head()

    #preds = aml.leader.predict()

In [109]:
#vf_pre_all_pd = generate_pandas_prepaid(vf_pre.drop('msisdn', 'partitioned_month'))

In [110]:
#vf_pre_all_pd.dtypes
#print vf_pre_all_pd.columns.values.tolist()

In [111]:
print vf_pre.rdd.getNumPartitions()
vf_pre.coalesce(1).write.mode('overwrite').format('parquet').save('/tmp/bbergua/tmp/vf_pre_all-'+month)

In [117]:
# $ hdfs getconf -confKey fs.defaultFS
# hdfs://nameservice1
# $ hdfs getconf -namenodes
# vgddp350hr.dc.sedc.internal.vodafone.com vgddp351hr.dc.sedc.internal.vodafone.com
out = subprocess.check_output('hdfs getconf -namenodes', shell=True)
nodes = ['hdfs://'+n for n in out.strip().split(' ')]

In [118]:
path_vf_pre_all = [n+'/tmp/bbergua/tmp/vf_pre_all-'+month for n in nodes]
path_vf_pre_all

['hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/tmp/bbergua/tmp/vf_pre_all-201712',
 'hdfs://vgddp351hr.dc.sedc.internal.vodafone.com/tmp/bbergua/tmp/vf_pre_all-201712']

In [119]:
i=0
from h2o.backend.connection import H2OServerError
while i < len(nodes):
    try:
        vf_pre_all_df_h2o = h2o.import_file(path=path_vf_pre_all[i])
        print vf_pre_all_df_h2o.nrows
        break
    except H2OServerError as e:
        print 'ERROR in', path_vf_pre_all[i]
        i=i+1
        #print e
#subprocess.call('hdfs dfs -rm -r /tmp/bbergua/tmp/vf_pre_all-'+month, shell=True)

ERROR in hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/tmp/bbergua/tmp/vf_pre_all-201712
Parse progress: |█████████████████████████████████████████████████████████| 100%
2730651


In [120]:
h2o.ls()

,key
0,AutoML_20180409_095222
1,AutoML_20180409_103218
2,AutoML_Feedback_AutoML_20180409_095222
3,AutoML_Feedback_AutoML_20180409_103218
4,AutoML_Leaderboard_automl_py_48_sid_88f9
5,AutoML_Leaderboard_automl_py_8_sid_88f9
6,DRF_0_AutoML_20180409_095222
7,DRF_0_AutoML_20180409_095222_cv_1
8,DRF_0_AutoML_20180409_095222_cv_2
9,DRF_0_AutoML_20180409_095222_cv_3


In [124]:
vf_pre_all_df_h2o['AC_fx_1llamada'] = vf_pre_all_df_h2o['AC_fx_1llamada'].asnumeric()
vf_pre_all_df_h2o['AC_fx_activacion'] = vf_pre_all_df_h2o['AC_fx_activacion'].asnumeric()
vf_pre_all_df_h2o['AC_fx_ultima_recarga'] = vf_pre_all_df_h2o['AC_fx_ultima_recarga'].asnumeric()
vf_pre_all_df_h2o['AC_x_fecha_nacimiento'] = vf_pre_all_df_h2o['AC_x_fecha_nacimiento'].asnumeric()
vf_pre_all_df_h2o['AC_fecha_beneficio_activo'] = vf_pre_all_df_h2o['AC_fecha_beneficio_activo'].asnumeric()

vf_pre_all_df_h2o['AC_num_prepago'] = vf_pre_all_df_h2o['AC_num_prepago'].asnumeric()
vf_pre_all_df_h2o['AC_num_pospago'] = vf_pre_all_df_h2o['AC_num_pospago'].asnumeric()
vf_pre_all_df_h2o['AC_num_total'] = vf_pre_all_df_h2o['AC_num_total'].asnumeric()
vf_pre_all_df_h2o['AC_min_llam_ultmes'] = vf_pre_all_df_h2o['AC_min_llam_ultmes'].asnumeric()
vf_pre_all_df_h2o['AC_num_sms_ultmes'] = vf_pre_all_df_h2o['AC_num_sms_ultmes'].asnumeric()
vf_pre_all_df_h2o['AC_ult3meses_total'] = vf_pre_all_df_h2o['AC_ult3meses_total'].asnumeric()
vf_pre_all_df_h2o['AC_media_ult3meses'] = vf_pre_all_df_h2o['AC_media_ult3meses'].asnumeric()
vf_pre_all_df_h2o['AC_diasdesdeultrecarga'] = vf_pre_all_df_h2o['AC_diasdesdeultrecarga'].asnumeric()
vf_pre_all_df_h2o['AC_numrecargasult3meses_total'] = vf_pre_all_df_h2o['AC_numrecargasult3meses_total'].asnumeric()

#vf_pre_all_df_h2o[''] = vf_pre_all_df_h2o[''].asnumeric()

In [125]:
vf_pre_all_df_h2o.describe()

Rows:2730651
Cols:103




,msisdn,partitioned_month,AC_fecha_ejecucion,nif,AC_fx_1llamada,AC_fx_activacion,AC_codigo_plan_precios,AC_fx_ultima_recarga,AC_estado_servicio,AC_tipo_documento_cliente,AC_tipo_documento_comprador,AC_nacionalidad,AC_x_fecha_nacimiento,AC_tac_fac,AC_num_prepago,AC_num_pospago,AC_num_total,AC_min_llam_ultmes,AC_num_sms_ultmes,AC_ult3meses_total,AC_media_ult3meses,AC_diasdesdeultrecarga,AC_numrecargasult3meses_total,AC_codigo_postal,AC_cod_golden,AC_cobertura_4g,AC_lortad,AC_deuda,AC_flag_huella_ono,AC_flag_4g_aperturas,AC_flag_4g_nodos,AC_flag_huella_vf,AC_flag_huella_neba,AC_flag_huella_euskaltel,AC_flag_cobertura_adsl,AC_flag_beneficio_activo,AC_fecha_beneficio_activo,CCC_Pagar_menos,CCC_Incidencia_Provision_Neba,CCC_Incidencia_Provision_Fibra,CCC_Incidencia_Provision_DSL,CCC_Incidencia_Tecnica,CCC_Incidencia_SGI,CCC_Incidencia_Resto,CCC_Incidencia_Provision_Movil,CCC_Resultado_No_Aplica,CCC_Resultado_Informacion,CCC_Resultado_Solucionado,CCC_Resultado_Retenido,CCC_Resultado_No_Retenido,CCC_Resultado_Escalo,CCC_Resultado_Envio_tecnico,CCC_Resultado_Transferencia,CCC_Resultado_Abono,CCC_Resultado_Bajas,CCC_Resultado_Reclamacion,CCC_Desactivacion_BA_Movil_TV,CCC_Desactivacion_TV,CCC_Desactivacion_Movil,CCC_Desactivacion_Total,CCC_Desactivacion_NET,CCC_Desactivacion_Fijo,CCC_Desactivacion_USB,CCC_Desactivacion_Resto,CCC_Ofrecimiento,CCC_Transferencia,CCC_Cobro,CCC_Precios,CCC_Portabilidad_Inversa,CCC_Portabilidad,CCC_Informacion,CCC_Cierre,CCC_Alta,CCC_Factura,CCC_Averia_DSL,CCC_Averia_Fibra,CCC_Averia_TV,CCC_Averia_Resto,CCC_Averia_Neba,CCC_Averia_Modem_Router,CCC_Baja,CCC_Consulta_Tecnica_TV,CCC_Consulta_Tecnica_Fibra,CCC_Consulta_Tecnica_Neba,CCC_Consulta_Tecnica_DSL,CCC_Consulta_Tecnica_Movil,CCC_Consulta_Tecnica_Modem_Router,CCC_Consulta_Tecnica_Resto,CCC_Consulta_Ficha,CCC_Consulta_Resto,CCC_Productos_Voz,CCC_Productos_Datos,CCC_Productos_Resto,CCC_Provision_Neba,CCC_Provision_Fibra,CCC_Provision_DSL,CCC_Provision_Resto,CCC_Provision_Movil,TNPS01,TNPS,TNPS2DET,TNPS2PRO,TNPS4
type,string,enum,enum,string,int,int,enum,int,enum,enum,enum,enum,int,enum,int,int,int,int,int,int,int,int,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,enum,enum,enum,enum
mins,NaN,,,NaN,17530101.0,17530101.0,,17530101.0,,,,,0.0,,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,,,,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,
mean,NaN,,,NaN,19965682.4616,20141385.6089,,19793635.6881,,,,,41.0638433839,,1.52362568486,0.263266891302,1.78689257617,13220.6039578,85.9490509772,1994.65055549,1689.22380304,77.4486904405,21.6841830025,,,,,,,,,,,,,,4.40123509009,1.83106519288e-06,1.53809476202e-05,1.53809476202e-05,1.72120128131e-05,1.06201781187e-05,0.0,1.94092910445e-05,1.83106519288e-06,0.000507205058427,0.0191166868267,0.0103770126611,0.000408327538012,0.000219361610107,0.000314577000137,3.40578125875e-05,0.000819584780333,2.12403562374e-05,0.0,0.000166260719513,4.35793515905e-05,2.27052083917e-05,0.000631717491543,0.0,9.99761595312e-05,3.69875168962e-05,1.79444388902e-05,1.06201781187e-05,0.000116821959306,0.0112313876801,0.000454104167834,2.19727823145e-06,1.4648521543e-06,0.00261585973455,0.0138142882412,0.00147693718458,0.0024008926809,0.00102136816459,0.000204713088564,0.00011828681146,4.61428428605e-05,0.000195191549561,4.65090558991e-05,5.12698254006e-06,0.000262940961697,1.2085030273e-05,3.69875168962e-05,6.59183469436e-06,5.0903612362e-05,0.00214490976694,4.43117776677e-05,2.34376344689e-05,1.09863911573e-06,1.09863911573e-06,3.66213038576e-06,1.79444388902e-05,0.019373402166

In [180]:
vf_pre_all_df_h2o_predict = vf_pre_all_df_h2o.drop(['TNPS01', 'TNPS2DET', 'TNPS2PRO', 'TNPS', 'TNPS4'])
vf_pre_all_preds = aml.predict(vf_pre_all_df_h2o_predict)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [181]:
#vf_pre_all_preds.head()

Append MSISDN, NIF and actual labels to predictions

In [182]:
vf_pre_all_preds_msisdn = vf_pre_all_preds.cbind(vf_pre_all_df_h2o[['msisdn', 'nif', 'TNPS01', label_preds, label_model]])
vf_pre_all_preds_msisdn

predict,DETRACTOR,NON DETRACTOR,msisdn,nif,TNPS01,TNPS,TNPS2DET
NON DETRACTOR,0.429564,0.570436,600006000,06754428C,nan,,
NON DETRACTOR,0.59915,0.40085,600009265,10577906N,nan,,
NON DETRACTOR,0.326921,0.673079,600013939,77470442T,nan,,
NON DETRACTOR,0.570642,0.429358,600014045,84144071V,nan,,
NON DETRACTOR,0.438054,0.561946,600017215,L7403214K,nan,,
NON DETRACTOR,0.418074,0.581926,600036363,11272337N,nan,,
NON DETRACTOR,0.538727,0.461273,600037750,0709620,nan,,
NON DETRACTOR,0.544664,0.455336,600055070,M8898863I,nan,,
NON DETRACTOR,0.529528,0.470472,600063018,867234945,nan,,
NON DETRACTOR,0.483511,0.516489,600069074,K9892729A,nan,,


Count number of rows and proportion of predictions

In [183]:
vf_pre_all_preds_nrows = vf_pre_all_preds_msisdn.nrows
vf_pre_all_preds_groupby = vf_pre_all_preds_msisdn.group_by(by='predict').count().get_frame()
proportion = 100*vf_pre_all_preds_groupby['nrow']/vf_pre_all_preds_nrows
proportion.set_names(['proportion'])
vf_pre_all_preds_groupby.cbind(proportion)

predict,nrow,proportion
DETRACTOR,2353,0.0861699
NON DETRACTOR,2.7283e+06,99.9138


As proportions of predictions are highly unbalanced, let's create a new prediction column (**predict2**) using a threshold on scoring column

In [184]:
#th_max_acc = test_metrics.accuracy()[0][0]
th_max_acc = 0.5
vf_pre_all_preds_msisdn[vf_pre_all_preds_msisdn["NON DETRACTOR"] >= th_max_acc, "predict2"] = "NON DETRACTOR"
vf_pre_all_preds_msisdn[vf_pre_all_preds_msisdn["NON DETRACTOR"] < th_max_acc, "predict2"] = "DETRACTOR"
vf_pre_all_preds_msisdn['predict2'] = vf_pre_all_preds_msisdn['predict2'].asfactor()
vf_pre_all_preds_msisdn

predict,DETRACTOR,NON DETRACTOR,msisdn,nif,TNPS01,TNPS,TNPS2DET,predict2
NON DETRACTOR,0.429564,0.570436,600006000,06754428C,nan,,,NON DETRACTOR
NON DETRACTOR,0.59915,0.40085,600009265,10577906N,nan,,,DETRACTOR
NON DETRACTOR,0.326921,0.673079,600013939,77470442T,nan,,,NON DETRACTOR
NON DETRACTOR,0.570642,0.429358,600014045,84144071V,nan,,,DETRACTOR
NON DETRACTOR,0.438054,0.561946,600017215,L7403214K,nan,,,NON DETRACTOR
NON DETRACTOR,0.418074,0.581926,600036363,11272337N,nan,,,NON DETRACTOR
NON DETRACTOR,0.538727,0.461273,600037750,0709620,nan,,,DETRACTOR
NON DETRACTOR,0.544664,0.455336,600055070,M8898863I,nan,,,DETRACTOR
NON DETRACTOR,0.529528,0.470472,600063018,867234945,nan,,,DETRACTOR
NON DETRACTOR,0.483511,0.516489,600069074,K9892729A,nan,,,NON DETRACTOR


Compare real label with this **predict2**  column newly created

In [185]:
vf_pre_all_preds_nrows = vf_pre_all_preds_msisdn.nrows
vf_pre_all_preds_groupby = vf_pre_all_preds_msisdn.group_by(by=['TNPS2DET', 'predict2']).count().get_frame()
proportion = 100*vf_pre_all_preds_groupby['nrow']/vf_pre_all_preds_nrows
proportion.set_names(['proportion'])
vf_pre_all_preds_groupby.cbind(proportion)

TNPS2DET,predict2,nrow,proportion
,NON DETRACTOR,1.53829e+06,56.3342
,DETRACTOR,1.14335e+06,41.8708
DETRACTOR,DETRACTOR,3472,0.127149
DETRACTOR,NON DETRACTOR,8327,0.304946
NON DETRACTOR,DETRACTOR,7284,0.26675
NON DETRACTOR,NON DETRACTOR,29932,1.09615


Count number of rows and proportion of predictions using the new **predict2** column

In [186]:
vf_pre_all_preds_nrows = vf_pre_all_preds_msisdn.nrows
vf_pre_all_preds_groupby = vf_pre_all_preds_msisdn.group_by(by='predict2').count().get_frame()
proportion = 100*vf_pre_all_preds_groupby['nrow']/vf_pre_all_preds_nrows
proportion.set_names(['proportion'])
vf_pre_all_preds_groupby.cbind(proportion)

predict2,nrow,proportion
DETRACTOR,1.1541e+06,42.2647
NON DETRACTOR,1.57655e+06,57.7353


As Prepaid data is by MSISDN (instead of NIF), then we need to group by NIF and aggregate scorings

In [415]:
vf_pre_all_preds_msisdn['nif'] = vf_pre_all_preds_msisdn['nif'].asfactor()
vf_pre_all_preds_id = vf_pre_all_preds_msisdn.group_by(by='nif').min(['NON DETRACTOR', 'TNPS01']).max(['DETRACTOR']).get_frame()
vf_pre_all_preds_id = vf_pre_all_preds_id[vf_pre_all_preds_id['nif'] != '']
#vf_pre_all_preds_id[(vf_pre_all_preds_id['min_TNPS01'] != None)].show()
#vf_pre_all_preds_id[(vf_pre_all_preds_id['min_TNPS01'] != vf_pre_all_preds_id['max_TNPS01'])].show()
vf_pre_all_preds_id.set_names(['nif', 'TNPS01', 'DETRACTOR', 'NON DETRACTOR'])
#vf_pre_all_preds_id
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] != None)].show()

nif,TNPS01,DETRACTOR,NON DETRACTOR
O62278511,1,0.325979,0.674021
00000398D,10,0.38746,0.61254
00001717R,10,0.437993,0.562007
00005606Y,8,0.485624,0.514376
00006283X,10,0.511998,0.488002
00008950P,8,0.555096,0.444904
00011821K,10,0.446366,0.553634
00014405D,10,0.506025,0.493975
00016419M,9,0.375972,0.624028
00017562E,8,0.346366,0.653634


In [391]:
vf_pre_all_preds_id.types

{u'DETRACTOR': u'real',
 u'NON DETRACTOR': u'real',
 u'TNPS01': u'int',
 u'nif': u'enum'}

In [406]:
#h2o.download_all_logs(dirname='/var/SP/data/home/bbergua/h2ologs', filename = 'autoh2o_log.zip')

Writing H2O logs to /var/SP/data/home/bbergua/h2ologs/autoh2o_log.zip


'/var/SP/data/home/bbergua/h2ologs/autoh2o_log.zip'

In [416]:
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] == None), 'TNPS01'] = -1

# H2O does not allow empty strings '' here, so we need to use 'NA' instead
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] <= -1), 'TNPS4'] = 'NA'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] >= 0) & (vf_pre_all_preds_id['TNPS01'] <= 3), 'TNPS4'] = 'HARD DETRACTOR'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] >= 4) & (vf_pre_all_preds_id['TNPS01'] <= 6), 'TNPS4'] = 'SOFT DETRACTOR'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] >= 7) & (vf_pre_all_preds_id['TNPS01'] <= 8), 'TNPS4'] = 'NEUTRAL'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] >= 9) & (vf_pre_all_preds_id['TNPS01'] <= 10), 'TNPS4'] = 'PROMOTER'

vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS4'] == 'NA'), 'TNPS'] = 'NA'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS4'] == 'HARD DETRACTOR') | (vf_pre_all_preds_id['TNPS4'] == 'SOFT DETRACTOR'), 'TNPS'] = 'DETRACTOR'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS4'] == 'NEUTRAL'), 'TNPS'] = 'NEUTRAL'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS4'] == 'PROMOTER'), 'TNPS'] = 'PROMOTER'

vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS'] != 'DETRACTOR'), 'TNPS2DET'] = 'NON DETRACTOR'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS'] == 'DETRACTOR'), 'TNPS2DET'] = 'DETRACTOR'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS'] == 'NA'), 'TNPS2DET'] = 'NA'

vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS'] != 'PROMOTER'), 'TNPS2PRO'] = 'NON PROMOTER'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS'] == 'PROMOTER'), 'TNPS2PRO'] = 'PROMOTER'
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS'] == 'NA'), 'TNPS2PRO'] = 'NA'

vf_pre_all_preds_id['TNPS2DET'] = vf_pre_all_preds_id['TNPS2DET'].asfactor()
vf_pre_all_preds_id['TNPS2PRO'] = vf_pre_all_preds_id['TNPS2PRO'].asfactor()
vf_pre_all_preds_id['TNPS'] = vf_pre_all_preds_id['TNPS'].asfactor()
vf_pre_all_preds_id['TNPS4'] = vf_pre_all_preds_id['TNPS4'].asfactor()

vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] != -1)].show()

nif,TNPS01,DETRACTOR,NON DETRACTOR,TNPS4,TNPS,TNPS2DET,TNPS2PRO
O62278511,1,0.325979,0.674021,HARD DETRACTOR,DETRACTOR,DETRACTOR,NON PROMOTER
00000398D,10,0.38746,0.61254,PROMOTER,PROMOTER,NON DETRACTOR,PROMOTER
00001717R,10,0.437993,0.562007,PROMOTER,PROMOTER,NON DETRACTOR,PROMOTER
00005606Y,8,0.485624,0.514376,NEUTRAL,NEUTRAL,NON DETRACTOR,NON PROMOTER
00006283X,10,0.511998,0.488002,PROMOTER,PROMOTER,NON DETRACTOR,PROMOTER
00008950P,8,0.555096,0.444904,NEUTRAL,NEUTRAL,NON DETRACTOR,NON PROMOTER
00011821K,10,0.446366,0.553634,PROMOTER,PROMOTER,NON DETRACTOR,PROMOTER
00014405D,10,0.506025,0.493975,PROMOTER,PROMOTER,NON DETRACTOR,PROMOTER
00016419M,9,0.375972,0.624028,PROMOTER,PROMOTER,NON DETRACTOR,PROMOTER
00017562E,8,0.346366,0.653634,NEUTRAL,NEUTRAL,NON DETRACTOR,NON PROMOTER


In [417]:
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] == -1)].show()

nif,TNPS01,DETRACTOR,NON DETRACTOR,TNPS4,TNPS,TNPS2DET,TNPS2PRO
G0475215P,-1,0.416053,0.583947,NA,NA,NA,NA
166867538,-1,0.521175,0.478825,NA,NA,NA,NA
1855209326,-1,0.444169,0.555831,NA,NA,NA,NA
20497392A,-1,0.51382,0.48618,NA,NA,NA,NA
30296569O,-1,0.396315,0.603685,NA,NA,NA,NA
375636270,-1,0.427803,0.572197,NA,NA,NA,NA
588673,-1,0.552913,0.447087,NA,NA,NA,NA
60846858O,-1,0.414228,0.585772,NA,NA,NA,NA
65562000E,-1,0.472199,0.527801,NA,NA,NA,NA
B1577102-T,-1,0.448444,0.551556,NA,NA,NA,NA


In [418]:
vf_pre_all_preds_id.types

{u'DETRACTOR': u'real',
 u'NON DETRACTOR': u'real',
 u'TNPS': u'enum',
 u'TNPS01': u'int',
 u'TNPS2DET': u'enum',
 u'TNPS2PRO': u'enum',
 u'TNPS4': u'enum',
 u'nif': u'enum'}

In [419]:
#vf_pre_all_preds_msisdn['nif'] = vf_pre_all_preds_msisdn['nif'].asfactor()
#vf_pre_all_preds_id = vf_pre_all_preds_msisdn.group_by(by='nif').mean(['DETRACTOR', 'NON DETRACTOR', 'TNPS01']).min(['TNPS01']).max(['TNPS01']).get_frame()
#vf_pre_all_preds_id.set_names(['nif', 'TNPS01', 'DETRACTOR', 'NON DETRACTOR'])
vf_pre_all_preds_id[vf_pre_all_preds_id["NON DETRACTOR"] >= vf_pre_all_preds_id["DETRACTOR"], "predict"] = "NON DETRACTOR"
vf_pre_all_preds_id[vf_pre_all_preds_id["NON DETRACTOR"] < vf_pre_all_preds_id["DETRACTOR"], "predict"] = "DETRACTOR"
vf_pre_all_preds_id['predict'] = vf_pre_all_preds_id['predict'].asfactor()
#vf_pre_all_preds_id['suma'] = vf_pre_all_preds_id.sum(False, 1, ['DETRACTOR', 'NON DETRACTOR'])
#vf_pre_all_preds_id
vf_pre_all_preds_id[(vf_pre_all_preds_id['TNPS01'] != None)]

nif,TNPS01,DETRACTOR,NON DETRACTOR,TNPS4,TNPS,TNPS2DET,TNPS2PRO,predict
G0475215P,-1,0.416053,0.583947,NA,NA,NA,NA,NON DETRACTOR
166867538,-1,0.521175,0.478825,NA,NA,NA,NA,DETRACTOR
1855209326,-1,0.444169,0.555831,NA,NA,NA,NA,NON DETRACTOR
20497392A,-1,0.51382,0.48618,NA,NA,NA,NA,DETRACTOR
30296569O,-1,0.396315,0.603685,NA,NA,NA,NA,NON DETRACTOR
375636270,-1,0.427803,0.572197,NA,NA,NA,NA,NON DETRACTOR
588673,-1,0.552913,0.447087,NA,NA,NA,NA,DETRACTOR
60846858O,-1,0.414228,0.585772,NA,NA,NA,NA,NON DETRACTOR
65562000E,-1,0.472199,0.527801,NA,NA,NA,NA,NON DETRACTOR
B1577102-T,-1,0.448444,0.551556,NA,NA,NA,NA,NON DETRACTOR


Count number of rows and proportion of predictions

In [420]:
vf_pre_all_preds_nrows = vf_pre_all_preds_id.nrows
vf_pre_all_preds_groupby = vf_pre_all_preds_id.group_by(by='predict').count().get_frame()
proportion = 100*vf_pre_all_preds_groupby['nrow']/vf_pre_all_preds_nrows
proportion.set_names(['proportion'])
vf_pre_all_preds_groupby.cbind(proportion)

predict,nrow,proportion
DETRACTOR,1.02263e+06,45.1091
NON DETRACTOR,1.24439e+06,54.8909


As proportions of predictions are highly unbalanced, let's create a new prediction column (**predict2**) using a threshold on scoring column

In [421]:
#th_max_acc = test_metrics.accuracy()[0][0]
th_max_acc = 0.5
vf_pre_all_preds_id[vf_pre_all_preds_id["NON DETRACTOR"] >= th_max_acc, "predict2"] = "NON DETRACTOR"
vf_pre_all_preds_id[vf_pre_all_preds_id["NON DETRACTOR"] < th_max_acc, "predict2"] = "DETRACTOR"
vf_pre_all_preds_id['predict2'] = vf_pre_all_preds_id['predict2'].asfactor()
vf_pre_all_preds_id

nif,TNPS01,DETRACTOR,NON DETRACTOR,TNPS4,TNPS,TNPS2DET,TNPS2PRO,predict,predict2
G0475215P,-1,0.416053,0.583947,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR
166867538,-1,0.521175,0.478825,NA,NA,NA,NA,DETRACTOR,DETRACTOR
1855209326,-1,0.444169,0.555831,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR
20497392A,-1,0.51382,0.48618,NA,NA,NA,NA,DETRACTOR,DETRACTOR
30296569O,-1,0.396315,0.603685,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR
375636270,-1,0.427803,0.572197,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR
588673,-1,0.552913,0.447087,NA,NA,NA,NA,DETRACTOR,DETRACTOR
60846858O,-1,0.414228,0.585772,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR
65562000E,-1,0.472199,0.527801,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR
B1577102-T,-1,0.448444,0.551556,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR


In [435]:
vf_pre_all_preds_id[vf_pre_all_preds_id['predict'] != vf_pre_all_preds_id['predict2']].show()

Count number of rows and proportion of predictions using the new **predict2** column

In [422]:
vf_pre_all_preds_nrows = vf_pre_all_preds_id.nrows
vf_pre_all_preds_groupby = vf_pre_all_preds_id.group_by(by='predict2').count().get_frame()
proportion = 100*vf_pre_all_preds_groupby['nrow']/vf_pre_all_preds_nrows
proportion.set_names(['proportion'])
vf_pre_all_preds_groupby.cbind(proportion)

predict2,nrow,proportion
DETRACTOR,1.02263e+06,45.1091
NON DETRACTOR,1.24439e+06,54.8909


As what we really need is to provide a prediction of the type ['PROMOTER', 'NEUTRAL', 'DETRACTOR'], we calculate cut points in predictions to preserve original proportions

In [423]:
print 'nrows:', vf_pre_all_preds_nrows
#print 'proportions', vf_pre_proportions

if 'HARD DETRACTOR' in vf_pre_proportions.keys() and 'SOFT DETRACTOR' in vf_pre_proportions.keys():
    vf_pre_proportions['DETRACTOR'] = vf_pre_proportions['HARD DETRACTOR'] + vf_pre_proportions['SOFT DETRACTOR']
    del vf_pre_proportions['HARD DETRACTOR']
    del vf_pre_proportions['SOFT DETRACTOR']

vf_pre_counts = {}
for k in vf_pre_proportions.keys():
    vf_pre_counts[k] = int(vf_pre_all_preds_nrows*vf_pre_proportions[k]/100)
diff = vf_pre_all_preds_nrows - sum(vf_pre_counts.values())
min_key = min(vf_pre_counts, key=vf_pre_counts.get) # Get the key with minimum value
vf_pre_counts[min_key] = vf_pre_counts[min_key] + diff # Assign rest to key with minimum num rows
print 'counts:', vf_pre_counts

ini = 0
end = 0
vf_pre_ranges = {}
for k in ['DETRACTOR', 'NEUTRAL', 'PROMOTER']: # It is necessary to force the order
    if end > 0:
        ini = end + 1
    end = ini + vf_pre_counts[k] - 1
    vf_pre_ranges[k] = [ini, end]
    #print k, ini, end
print 'ranges:', vf_pre_ranges

nrows: 2267021
counts: {u'NEUTRAL': 328896, u'PROMOTER': 1392403, 'DETRACTOR': 545722}
ranges: {'NEUTRAL': [545722, 874617], 'PROMOTER': [874618, 2267020], 'DETRACTOR': [0, 545721]}


Using the cut points calculated above, create a new **predict3** column with final predictions

In [424]:
for k in vf_pre_ranges.keys():
    r = range(vf_pre_ranges[k][0], vf_pre_ranges[k][1]+1, 1)
    print k, r[0], r[-1]
    vf_pre_all_preds_id[r, 'predict3'] = k
    #tmp = vf_pre_all_preds_id[r, :]
    #print tmp.head()
vf_pre_all_preds_id['predict3'] = vf_pre_all_preds_id['predict3'].asfactor()
vf_pre_all_preds_id

NEUTRAL 545722 874617
PROMOTER 874618 2267020
DETRACTOR 0 545721


nif,TNPS01,DETRACTOR,NON DETRACTOR,TNPS4,TNPS,TNPS2DET,TNPS2PRO,predict,predict2,predict3
G0475215P,-1,0.416053,0.583947,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR,DETRACTOR
166867538,-1,0.521175,0.478825,NA,NA,NA,NA,DETRACTOR,DETRACTOR,DETRACTOR
1855209326,-1,0.444169,0.555831,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR,DETRACTOR
20497392A,-1,0.51382,0.48618,NA,NA,NA,NA,DETRACTOR,DETRACTOR,DETRACTOR
30296569O,-1,0.396315,0.603685,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR,DETRACTOR
375636270,-1,0.427803,0.572197,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR,DETRACTOR
588673,-1,0.552913,0.447087,NA,NA,NA,NA,DETRACTOR,DETRACTOR,DETRACTOR
60846858O,-1,0.414228,0.585772,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR,DETRACTOR
65562000E,-1,0.472199,0.527801,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR,DETRACTOR
B1577102-T,-1,0.448444,0.551556,NA,NA,NA,NA,NON DETRACTOR,NON DETRACTOR,DETRACTOR


Count number of rows and proportion of predictions using the final **predict3** column, that must match original proportions

In [425]:
vf_pre_all_preds_nrows = vf_pre_all_preds_id.nrows
vf_pre_all_preds_groupby = vf_pre_all_preds_id.group_by(by='predict3').count().get_frame()
proportion = 100*vf_pre_all_preds_groupby['nrow']/vf_pre_all_preds_nrows
proportion.set_names(['proportion'])
vf_pre_all_preds_groupby.cbind(proportion)

predict3,nrow,proportion
DETRACTOR,545722,24.0722
NEUTRAL,328896,14.5078
PROMOTER,1.3924e+06,61.4199


Convert predictions to Pandas, to bring predictions back to Spark again

In [426]:
print time.ctime()
start = time.time()
print 'Converting predictions to Pandas'
vf_pre_all_preds_id_pd = vf_pre_all_preds_id[['nif', label_preds, 'predict3']].as_data_frame()
#vf_pre_all_preds_id_pandas
end = time.time()
print 'Process took:', "{0:.2f}".format((end - start)/60), 'minutes'

Tue Apr 10 10:31:44 2018
Converting predictions to Pandas
Process took: 0.11 minutes


Create a Spark DataFrame from the Pandas object created above

In [430]:
#Create PySpark DataFrame Schema
pd_schema = StructType([StructField('nif',        StringType(),True),
                        StructField('label_preds',StringType(),True),
                        StructField('predict',    StringType(),True)])

#Create Spark DataFrame from Pandas
df_vf_pre_preds = sqlContext.createDataFrame(vf_pre_all_preds_id_pd, pd_schema)
#Important to order columns in the same order as the target database
df_vf_pre_preds = df_vf_pre_preds.select('nif', 'label_preds', 'predict')

Count number of rows of actual label and final predictions

In [431]:
print df_vf_pre_preds.count()
#df_vf_pre_preds.groupby('label_preds').count().show()
#print df_vf_pre_preds.filter('label_preds == "NaN"').count()
df_vf_pre_preds.groupby('predict').count().show()

2267021
+---------+-------+
|  predict|  count|
+---------+-------+
|  NEUTRAL| 328896|
| PROMOTER|1392403|
|DETRACTOR| 545722|
+---------+-------+



For those clients that we really have actual label, use actual label. Otherwise, use prediction.

In [432]:
vf_pre_preds = df_vf_pre_preds.withColumn('predict', 
                                          when(df_vf_pre_preds['label_preds'] != "NaN",
                                               df_vf_pre_preds['label_preds'])\
                                         .otherwise(df_vf_pre_preds['predict']))\
                              .drop('label_preds')
#print vf_pre_preds.count()
#vf_pre_preds.groupby('predict').count().show()

vf_pre_preds_count = vf_pre_preds.count()
print vf_pre_preds_count
vf_pre_preds_groupby = vf_pre_preds.groupby('predict').count()
vf_pre_preds_proportions = vf_pre_preds_groupby.withColumn('proportion', 100*vf_pre_preds_groupby['count']/vf_pre_preds_count).cache()
vf_pre_preds_proportions.withColumn('proportion', bround(vf_pre_preds_proportions['proportion'], 2)).show()

2267021
+---------+-------+----------+
|  predict|  count|proportion|
+---------+-------+----------+
|  NEUTRAL| 328991|     14.51|
| PROMOTER|1392814|     61.44|
|DETRACTOR| 545216|     24.05|
+---------+-------+----------+



In [434]:
print vf_pre_preds.select('nif').distinct().count()

2267021


Write predictions to HDFS

In [433]:
vf_pre_preds.write.mode('overwrite').format('parquet').save('/tmp/bbergua/tmp/vf_pre_preds-'+month)

In [75]:
#vf_pre_preds = spark.read.parquet('/tmp/bbergua/tmp/vf_pre_preds-'+month)

## Oracle Non-Prepaid segments

### Load Oracle Postpaid

In [436]:
##print 'ACC CAR rows:', vf_pos.filter('partitioned_month == "%s"' % month).count()
#print 'Accenture CAR distinct MSISDN:', spark.table('udf_es.par_explic_lin_6m_' + month).select('msisdn').distinct().count()
#print 'Accenture CAR distinct NIF:   ', spark.table('udf_es.par_explic_cli_6m_' + month).select('nif').distinct().count()
##print 'AC_FINAL_POS rows:', spark.table('raw_es.vf_pos_ac_final').filter('partitioned_month == "%s"' % month).count()
#print 'AC_FINAL_POSP distinct MSISDN:', spark.table('raw_es.vf_pos_ac_final').filter('partitioned_month == "%s"' % month).select('x_id_red').distinct().count()
#print 'AC_FINAL_POSP distinct NIF:   ', spark.table('raw_es.vf_pos_ac_final').filter('partitioned_month == "%s"' % month).select('x_num_ident').distinct().count()

In [437]:
#spark.table('raw_es.vf_pos_ac_final').printSchema()print time.ctime()

In [16]:
if '/var/SP/data/home/bbergua/fy17.capsule/convergence/src/main/python' not in sys.path:
    print 'Adding [...]/convergence/src/main/python to sys.path'
    sys.path.append('/var/SP/data/home/bbergua/fy17.capsule/convergence/src/main/python')

from configuration import Configuration
from DP_prepare_input_cvm_pospago import DPPrepareInputCvmPospago

#del ac_postpaid
if not 'ac_postpaid' in globals():
    print time.ctime()
    start = time.time()
    
    conf = Configuration(sc)
    ac_postpaid = DPPrepareInputCvmPospago(conf, month)
    ac_postpaid.calculate_mobile_only_and_convergent_by_id()
    
    end = time.time()
    print 'Process took:', "{0:.2f}".format((end - start)/60), 'minutes'

Tue Apr 10 14:19:55 2018
SparkContext already loaded. master is yarn
Removing spurious header in raw_es.vf_pos_ac_final ...
Removing corrupt rows from raw_es.vf_pos_ac_final ...
Finished generate_features()
Making 194490 mobile-only clients convergent
Making 194490 non-convergent clients convergent
Finished set_convergent_ids()
Finished calculate_mobile_only_and_convergent_by_id()
Process took: 9.18 minutes


In [440]:
#print ac_postpaid.dataById.filter('partitioned_month == "%s"' % month).count()
#print ac_postpaid.dataById.filter('partitioned_month == "%s"' % month).select('x_num_ident').distinct().count()

3290364
3285656


In [443]:
#print ac_postpaid.dataById.filter('partitioned_month == "%s"' % month).distinct().count()

3285656


In [441]:
#ac_postpaid.dataById.filter('partitioned_month == "%s"' % month).groupby('x_num_ident').count().sort('count', ascending=False).show()

+-----------+-----+
|x_num_ident|count|
+-----------+-----+
|  14788947G|    4|
|  94841368R|    3|
|  12325272Y|    3|
|  16552274I|    3|
|  02438499Q|    3|
|  19294348U|    3|
|  N9131075Z|    3|
|  34702948D|    3|
|  43970668J|    3|
|  55612284L|    3|
|  13160056H|    3|
|  05123249T|    3|
|  14723519U|    3|
|  50035787D|    3|
|  72627840X|    3|
|  Z8363564W|    3|
|  I3160786X|    3|
|  21963355X|    3|
|  50300247O|    3|
|  41109451F|    3|
+-----------+-----+
only showing top 20 rows



In [442]:
#ac_postpaid.dataById.filter('x_num_ident == "14788947G"').show()

+-----------+-----------------+------+--------+------+------------+-------+------+-----------------+----------------+---------------+--------+------------+-----------+----------------+------+---------+------------+----------+--------------+------------+--------------+-------------------+-------------+-------------+---------+---------+--------+--------------+--------------+-------------+----------+-----------+------------+-----------+--------------+------------+------------+------------------+----------------------+---------------------+------------------+--------------+-----------------+-------------+----------------+------------------+-----------------+----------+-----------------+----------------+----------------+---------------+-----------------+-------------------------+---------------+-----------+------------------------+---------------+------------------+----------------------+---------------------+-----------------------+---------------+---------------+--------------------+-----

### Oracle Non-Prepaid segment - Data Preparation

In [17]:
def prepare_dataset_postpaid():
    # Load Oracle AC Final Postpaid
    #del vf_pos
    #if not 'vf_pos' in globals():
    #print 'Load Oracle Postpaid CAR from Hive for month =', month
    #vf_pos = spark.table('udf_es.par_explic_cli_6m_' + month)
    print 'Load Oracle AC Final Postpaid from Hive for month =', month
    # Load AC Final Postpaid by Id
    vf_pos = ac_postpaid.dataById
    if 'partitioned_month' in vf_pos.columns:
        if month is not None:
            vf_pos = vf_pos.filter('partitioned_month == "%s"' % month)
        vf_pos.groupby('partitioned_month').count().show()
    #vf_pos = vf_pos.drop('x_sfid_cuenta', 'x_sfid_servicio', 'sfid_adsl', 'sfid_adslenprov', 
    #                     'sfid_ftth', 'sfid_hz', 'sfid_lpd', 'sfid_tv', 'sfid_futbol', 
    #                     'sfid_canje', 'sfid_cambio_pprecios', 'sfid_simo')
    vf_pos = vf_pos.withColumnRenamed('x_id_red', 'msisdn')
    vf_pos = vf_pos.withColumnRenamed('x_num_ident', 'nif')
    #vf_pos = vf_pos.withColumn('tipo_documento_cliente', when(vf_pos['tipo_documento_cliente'] == 'Tarj.Residente',  'Tarj. Residente')
    #                                                    .when(vf_pos['tipo_documento_cliente'] == 'NIE',             'Tarj. Residente')
    #                                                    .when(vf_pos['tipo_documento_cliente'] == 'T_RES',           'Tarj. Residente')
    #                                                    .when(vf_pos['tipo_documento_cliente'] == 'TARJ. RESIDENTE', 'Tarj. Residente')
    #                                                    .when(vf_pos['tipo_documento_cliente'] == 'PASAPORTE',       'Pasaporte')
    #                                                    .when(vf_pos['tipo_documento_cliente'] == 'GMT+1',           '')
    #                                                    .otherwise(vf_pos['tipo_documento_cliente']))
    vf_pos = add_prefix_columns(vf_pos, 'AC')
    #vf_pos = vf_pos.withColumnRenamed('nif', 'AC_nif')
    #vf_pos.printSchema()

    # Join Postpaid data with Call Centre Calls
    print 'Join Postpaid data with Call Centre Calls'
    vf_pos_ccc = vf_pos.join(ccc_id, ['nif', 'partitioned_month'], 'left_outer')
    ccc_columns = [x for x in ccc_id.columns if x not in ['nif', 'msisdn', 'partitioned_month']]
    vf_pos_ccc = vf_pos_ccc.fillna(0, ccc_columns)
    #vf_pos_ccc.groupby('partitioned_month').count().show()
    #vf_pos_ccc.printSchema()

    # Join Postpaid data with TNPS (to get the label)
    #tnps_nif.show()
    tnps_id_pospaid = tnps_id.filter('SEGMENTACION != "Prepaid"').select(['nif', 'partitioned_month', 'SEGMENTACION', 'TNPS01', 'TNPS', 'TNPS2DET', 'TNPS2PRO', 'TNPS4'])
    #tnps_id_pospaid.groupby('partitioned_month').count().show()
    print 'TNPS count:', tnps_id_pospaid.count()
    tnps_id_pospaid.groupby('TNPS4').count().show()
    print 'Join TNPS with Postpaid data'
    vf_pos_ccc_tnps = vf_pos_ccc.join(tnps_id_pospaid, ['nif', 'partitioned_month'], 'left_outer')
    #vf_pos_ccc_tnps.groupby('partitioned_month').count().show()
    
    # Now, generate dataset for model training
    vf_pos = vf_pos_ccc_tnps.drop('year', 'month', 'day')
    vf_pos = fix_column_names(vf_pos)

    # Calculate proportions of TNPS4 levels
    vf_pos_count = vf_pos.na.drop(subset='TNPS4').count()
    print 'vf_pos count:', vf_pos_count
    vf_pos_groupby = vf_pos.na.drop(subset='TNPS4').groupby('TNPS4').count()
    vf_pos_proportions = vf_pos_groupby.withColumn('proportion', 100*vf_pos_groupby['count']/vf_pos_count).cache()
    vf_pos_proportions_dict = dict(vf_pos_proportions.select('TNPS4', 'proportion').collect())
    vf_pos_proportions.withColumn('proportion', bround(vf_pos_proportions['proportion'], 2)).show()
    
    vf_pos.printSchema()
    
    return vf_pos, vf_pos_proportions_dict

In [18]:
vf_pos, vf_pos_proportions = prepare_dataset_postpaid()

Load Oracle AC Final Postpaid from Hive for month = 201712
+-----------------+-------+
|partitioned_month|  count|
+-----------------+-------+
|           201712|3285656|
+-----------------+-------+

Appending AC prefix to columns ...
Join Postpaid data with Call Centre Calls
TNPS count: 284737
+--------------+------+
|         TNPS4| count|
+--------------+------+
|HARD DETRACTOR| 37155|
|       NEUTRAL| 60803|
|      PROMOTER|154759|
|SOFT DETRACTOR| 32020|
+--------------+------+

Join TNPS with Postpaid data
vf_pos count: 284737
+--------------+------+----------+
|         TNPS4| count|proportion|
+--------------+------+----------+
|HARD DETRACTOR| 37155|     13.05|
|       NEUTRAL| 60803|     21.35|
|      PROMOTER|154759|     54.35|
|SOFT DETRACTOR| 32020|     11.25|
+--------------+------+----------+

root
 |-- nif: string (nullable = true)
 |-- partitioned_month: string (nullable = true)
 |-- AC_prodhz: string (nullable = true)
 |-- AC_prodadsl: string (nullable = true)
 |-- AC

In [19]:
vf_pos_proportions

{u'HARD DETRACTOR': 13.048883706718831,
 u'NEUTRAL': 21.354091670559146,
 u'PROMOTER': 54.351559509301566,
 u'SOFT DETRACTOR': 11.245465113420455}

### Oracle Non-Prepaid segment - Training

In [20]:
vf_pos_training, vf_pos_training_proportions = prepare_training_data(vf_pos)

Class count before cleaning:
+-------------+------+----------+
|     TNPS2DET| count|proportion|
+-------------+------+----------+
|NON DETRACTOR|215562|     75.71|
|    DETRACTOR| 69175|     24.29|
+-------------+------+----------+

fractions: {u'NON DETRACTOR': 0.3209053543760032, u'DETRACTOR': 1.0}
Class count after balancing:
+-------------+-----+----------+
|     TNPS2DET|count|proportion|
+-------------+-----+----------+
|    DETRACTOR|69175|      50.0|
|NON DETRACTOR|69181|      50.0|
+-------------+-----+----------+

root
 |-- partitioned_month: string (nullable = true)
 |-- AC_prodhz: string (nullable = true)
 |-- AC_prodadsl: string (nullable = true)
 |-- AC_x_plan: string (nullable = true)
 |-- AC_promocion_vf: string (nullable = true)
 |-- AC_prodlpd: string (nullable = true)
 |-- AC_puntos: integer (nullable = true)
 |-- AC_sistema_operativo: string (nullable = true)
 |-- AC_promocion_tarifa: string (nullable = true)
 |-- AC_flag_huella_von: string (nullable = true)
 |-- A

In [21]:
vf_pos_training_proportions

{u'DETRACTOR': 24.294348820139287, u'NON DETRACTOR': 75.70565117986071}

In [22]:
#vf_pos_training.show(1)

In [23]:
print time.ctime()
start = time.time()
print vf_pos_training.count()
vf_pos_training.write.mode('overwrite').format('parquet').save('/tmp/bbergua/tmp/vf_pos_training-'+month)
end = time.time()
print 'Process took:', "{0:.2f}".format((end - start)/60), 'minutes'

Tue Apr 10 16:09:49 2018
138356
Process took: 25.32 minutes


In [24]:
# $ hdfs getconf -confKey fs.defaultFS
# hdfs://nameservice1
# $ hdfs getconf -namenodes
# vgddp350hr.dc.sedc.internal.vodafone.com vgddp351hr.dc.sedc.internal.vodafone.com
out = subprocess.check_output('hdfs getconf -namenodes', shell=True)
nodes = ['hdfs://'+n for n in out.strip().split(' ')]

In [25]:
path_vf_pos_training = [n+'/tmp/bbergua/tmp/vf_pos_training-'+month for n in nodes]
path_vf_pos_training[0]

'hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/tmp/bbergua/tmp/vf_pos_training-201712'

In [26]:
vf_pos_training_df_h2o = h2o.import_file(path=path_vf_pos_training[0])
vf_pos_training_df_h2o.nrows
#subprocess.call('hdfs dfs -rm -r /tmp/bbergua/tmp/vf_pos_training-'+month, shell=True)

Parse progress: |█████████████████████████████████████████████████████████| 100%


138356

In [27]:
if False:
    vf_pos_training_pd = generate_pandas_data(vf_pos_training)
    # Copy to H2O

    #h2o.remove_all()
    #print balanced_pd.columns.values.tolist()
    #os.environ["PYTHONIOENCODING"] = "UTF-8"
    print 'Copying data to H2O'
    h2o.remove('vf_pos_training_df_h2o.hex')
    vf_pos_training_df_h2o = h2o.H2OFrame(vf_pos_training_pd, destination_frame='vf_pos_training_df_h2o.hex', header=1)
    vf_pos_training_df_h2o.head()

In [28]:
h2o.ls()

,key
0,part_00000_ee124499_36a1_40c4_a2ee_04852471cb6...


In [29]:
vf_pos_training_df_h2o.describe()

Rows:138356
Cols:281




,partitioned_month,AC_prodhz,AC_prodadsl,AC_x_plan,AC_promocion_vf,AC_prodlpd,AC_puntos,AC_sistema_operativo,AC_promocion_tarifa,AC_flag_huella_von,AC_prodtivo,AC_x_tipo_ident,AC_part_status,AC_pprecios_destino,AC_x_sexo,AC_plandatos,AC_ppid_destino,AC_cod_segfid,AC_modelo,AC_tarifa_canje,AC_x_nacionalidad,AC_flag_cobertura_adsl,AC_codigo_postal,AC_x_dia_emision,AC_x_subtipo,AC_prodvfbox,AC_prodftth,AC_prodadslenprov,AC_x_tipo_cliente,AC_sum_flagvfbox,AC_sum_flaghz,AC_sum_flagvoz,AC_sum_flagtivo,AC_sum_flaglpd,AC_sum_flagfutbol,AC_sum_flagftth,AC_sum_flagadsl,AC_sum_arpu,AC_sum_cantidad_pendiente,AC_sum_cuotas_pendientes,AC_is_mobile_only_new,AC_is_mobile_only,AC_is_convergent_new,AC_is_convergent,AC_max_cobertura_4g,AC_max_flag_huella_vf,AC_max_flag_ebilling,AC_max_lortad,AC_max_flag_financia,AC_max_vfsmartphone,AC_max_flag_nh_real,AC_max_seg_cliente,AC_max_flag_4g_nodos,AC_max_flag_existe_fbb_hogar,AC_max_flag_siebel,AC_max_flag_4g,AC_max_flag_huella_movistar,AC_max_terminal_4g,AC_max_flag_desc_conv,AC_max_flag_financia_simo,AC_max_flag_4g_aperturas,AC_max_flag_huella_jazztel,AC_max_terminalmms,AC_max_num_pospago,AC_max_flag_huella_neba,AC_max_flag_nh_prevista,AC_max_deuda,AC_max_cobertura_4g_plus,AC_max_flag_huella_ono,AC_max_num_prepago,AC_max_flag_cuenta_superintegral,AC_max_flag_huella_euskaltel,AC_max_num_total,AC_min_days_since_install_date,AC_min_days_since_fecha_fin_cp_tarifa,AC_min_days_since_x_fecha_nacimiento,AC_min_days_since_x_fecha_activacion,AC_min_days_since_fecha_fin_cp_vf,AC_min_days_since_fecha_alta_ss_ftth,AC_min_days_since_fecha_alta_ss_adsl,AC_min_days_since_fecha_alta_tivo,AC_min_days_since_fecha_alta_ss_hz,AC_min_days_since_fecha_alta_ss_adslenprov,AC_min_days_since_fecha_alta_vfbox,AC_min_days_since_fecha_ultima_financiacion,AC_min_days_since_fecha_alta_futbol,AC_min_days_since_fecha_alta_servicio,AC_min_days_since_fecha_transferencia,AC_min_days_since_fecha_antiguedad_cliente,AC_min_days_since_fecha_canje,AC_min_days_since_x_fecha_creacion_cuenta,AC_min_days_since_last_upd,AC_min_days_since_fecha_cambio,AC_min_days_since_x_fecha_recepcion,AC_min_days_since_x_fecha_ini_prov,AC_min_days_since_x_fecha_creacion_servicio,AC_min_x_dia_emision,AC_min_years_since_install_date,AC_min_years_since_fecha_fin_cp_tarifa,AC_min_years_since_x_fecha_nacimiento,AC_min_years_since_x_fecha_activacion,AC_min_years_since_fecha_fin_cp_vf,AC_min_years_since_fecha_alta_ss_ftth,AC_min_years_since_fecha_alta_ss_adsl,AC_min_years_since_fecha_alta_tivo,AC_min_years_since_fecha_alta_ss_hz,AC_min_years_since_fecha_alta_ss_adslenprov,AC_min_years_since_fecha_alta_vfbox,AC_min_years_since_fecha_ultima_financiacion,AC_min_years_since_fecha_alta_futbol,AC_min_years_since_fecha_alta_servicio,AC_min_years_since_fecha_transferencia,AC_min_years_since_fecha_antiguedad_cliente,AC_min_years_since_fecha_canje,AC_min_years_since_x_fecha_creacion_cuenta,AC_min_years_since_last_upd,AC_min_years_since_fecha_cambio,AC_min_years_since_x_fecha_recepcion,AC_min_years_since_x_fecha_ini_prov,AC_min_years_since_x_fecha_creacion_servicio,AC_avg_days_since_install_date,AC_avg_days_since_fecha_fin_cp_tarifa,AC_avg_days_since_x_fecha_nacimiento,AC_avg_days_since_x_fecha_activacion,AC_avg_days_since_fecha_fin_cp_vf,AC_avg_days_since_fecha_alta_ss_ftth,AC_avg_days_since_fecha_alta_ss_adsl,AC_avg_days_since_fecha_alta_tivo,AC_avg_days_since_fecha_alta_ss_hz,AC_avg_days_since_fecha_alta_ss_adslenprov,AC_avg_days_since_fecha_alta_vfbox,AC_avg_days_since_fecha_ultima_financiacion,AC_avg_days_since_fecha_alta_futbol,AC_avg_days_since_fecha_alta_servicio,AC_avg_days_since_fecha_transferencia,AC_avg_days_since_fecha_antiguedad_cliente,AC_avg_days_since_fecha_canje,AC_avg_days_since_x_fecha_creacion_cuenta,AC_avg_days_since_last_upd,AC_avg_days_since_fecha_cambio,AC_avg_days_since_x_fecha_recepcion,AC_avg_days_since_x_fecha_ini_prov,AC_avg_days_since_x_fecha_creacion_servicio,AC_avg_x_dia_emision,AC_avg_years_since_install_date,AC_avg_years_since_fecha_fin_cp_tarif

In [30]:
#data_df_h2o.types

In [31]:
# Split the data into Train/Test/Validation with Train having 70% and test and validation 15% each
train,test,valid = vf_pos_training_df_h2o.split_frame(ratios=[.7, .15])

In [32]:
# Identify predictors and response
#x = train.columns
y = label_model
#x.remove(y)
x = [item for item in train.columns if item not in ['msisdn', 'nif', 'TNPS01', 'TNPS2DET', 'TNPS2PRO', 'TNPS', 'TNPS4']]

In [33]:
train.types[y]

u'enum'

In [34]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
valid[y] = valid[y].asfactor()

In [35]:
train.types[y]

u'enum'

In [37]:
from h2o.automl import H2OAutoML
# Run AutoML for 60 seconds
aml = H2OAutoML(max_runtime_secs = 60, seed=1234)
aml.train(x = x, y = y,
          training_frame = train,
          leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [38]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb

model_id,auc,logloss
DRF_0_AutoML_20180410_164942,0.592199,0.679856
StackedEnsemble_AllModels_0_AutoML_20180410_164942,0.592184,0.679868
StackedEnsemble_BestOfFamily_0_AutoML_20180410_164942,0.592184,0.679868


In [39]:
# The leader model is stored here
aml.leader

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_0_AutoML_20180410_164942


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.246620204321
RMSE: 0.496608703428
LogLoss: 0.686637591279
Mean Per-Class Error: 0.447131467861
AUC: 0.575128795932
Gini: 0.150257591865
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.257488788052: 


,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,604.0,38149.0,0.9844,(38149.0/38753.0)
NON DETRACTOR,258.0,38488.0,0.0067,(258.0/38746.0)
Total,862.0,76637.0,0.4956,(38407.0/77499.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2574888,0.6671347,367.0
max f2,0.0405477,0.8333082,399.0
max f0point5,0.4260851,0.5667787,270.0
max accuracy,0.5100273,0.5528716,189.0
max precision,0.9340097,1.0,0.0
max recall,0.0405477,1.0,399.0
max specificity,0.9340097,1.0,0.0
max absolute_mcc,0.4696407,0.1074353,229.0
max min_per_class_accuracy,0.5026248,0.5501213,197.0
max mean_per_class_accuracy,0.5100273,0.5528685,189.0


Gains/Lift Table: Avg response rate: 50.00 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-04-10 16:53:07,3 min 20.523 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-04-10 16:53:08,3 min 21.445 sec,1.0,0.5710796,5.3017650,0.5330851,1.0217926,0.4981983,0.5739113,5.3760769,0.5274343,0.9829753,0.5016181
,2018-04-10 16:53:10,3 min 23.249 sec,2.0,0.5633280,4.5207792,0.5320447,1.0214381,0.4980371,0.5336069,1.7084646,0.5363241,1.0295098,0.5016181
,2018-04-10 16:53:15,3 min 28.261 sec,6.0,0.5371090,2.2636675,0.5376243,1.0208506,0.4996769,0.5060093,0.7194868,0.5548797,1.2403778,0.5016181
,2018-04-10 16:53:20,3 min 33.201 sec,13.0,0.5126108,0.9234059,0.5482026,1.0400939,0.5000517,0.4989278,0.6917752,0.5658210,1.2862138,0.4976627
,2018-04-10 16:53:27,3 min 40.199 sec,23.0,0.5025203,0.7089151,0.5602217,1.2517260,0.4964128,0.4960639,0.6854595,0.5771540,1.3376623,0.4966353
,2018-04-10 16:53:35,3 min 48.149 sec,34.0,0.4991695,0.6937076,0.5671836,1.2646304,0.4935160,0.4948015,0.6827607,0.5832114,1.3582418,0.4979196
,2018-04-10 16:53:44,3 min 56.849 sec,46.0,0.4970847,0.6877000,0.5733551,1.3007627,0.4935289,0.4943275,0.6817647,0.5857964,1.3685315,0.4949402
,2018-04-10 16:53:47,4 min 0.420 sec,50.0,0.4966087,0.6866376,0.5751288,1.3291523,0.4955806,0.4941644,0.6814267,0.5865971,1.4199800,0.4985360


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
AC_sistema_operativo,18618.1855469,1.0,0.0855072
AC_ppid_destino,15517.4912109,0.8334588,0.0712667
AC_plandatos,9306.8466797,0.4998794,0.0427433
AC_x_plan,8838.8134766,0.4747409,0.0405938
AC_pprecios_destino,6839.9912109,0.3673823,0.0314138
---,---,---,---
CCC_Consulta_Ficha,0.5662745,0.0000304,0.0000026
AC_max_flag_siebel,0.4444586,0.0000239,0.0000020
CCC_Productos_Voz,0.4102564,0.0000220,0.0000019
AC_x_subtipo,0.0,0.0,0.0



See the whole table with table.as_data_frame()


Model metrics of train dataset for max accuracy (if BinomialModel):

In [40]:
from h2o.model.metrics_base import H2OBinomialModelMetrics#, H2OMultinomialModelMetrics
train_metrics = aml.leader.model_performance()
if isinstance(train_metrics, H2OBinomialModelMetrics):
    #print aml.leader.confusion_matrix(metrics="accuracy")
    print train_metrics.confusion_matrix(metrics="accuracy")
    print 'AUC:', train_metrics.auc()
    print 'Accuracy:', train_metrics.accuracy()

Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.510027268324: 


,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,22763.0,15990.0,0.4126,(15990.0/38753.0)
NON DETRACTOR,18662.0,20084.0,0.4816,(18662.0/38746.0)
Total,41425.0,36074.0,0.4471,(34652.0/77499.0)



AUC: 0.575128795932
Accuracy: [[0.5100272683239445, 0.5528716499567736]]


Model metrics of test dataset, and confusion matrix for max accuracy (if BinomialModel):

In [41]:
#test_pred = aml.leader.predict(test)
test_metrics = aml.leader.model_performance(test_data=test)
print test_metrics


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.243427142234
RMSE: 0.49338336234
LogLoss: 0.679856088284
Mean Per-Class Error: 0.433031974694
AUC: 0.592198721966
Gini: 0.184397443932
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.328618249443: 


,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,329.0,9935.0,0.9679,(9935.0/10264.0)
NON DETRACTOR,145.0,10266.0,0.0139,(145.0/10411.0)
Total,474.0,20201.0,0.4875,(10080.0/20675.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3286182,0.6707174,357.0
max f2,0.1896887,0.8353392,396.0
max f0point5,0.4431816,0.5790450,269.0
max accuracy,0.4976830,0.5671100,206.0
max precision,0.8941962,1.0,0.0
max recall,0.1896887,1.0,396.0
max specificity,0.8941962,1.0,0.0
max absolute_mcc,0.4976830,0.1340421,206.0
max min_per_class_accuracy,0.5014615,0.5640101,201.0
max mean_per_class_accuracy,0.5005981,0.5669680,202.0


Gains/Lift Table: Avg response rate: 50.36 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100121,0.7028346,1.3910756,1.3910756,0.7004831,0.7004831,0.0139276,0.0139276,39.1075586,39.1075586
,2,0.0200242,0.6770226,1.3335138,1.3622947,0.6714976,0.6859903,0.0133513,0.0272788,33.3513837,36.2294712
,3,0.0300363,0.6608922,1.3239202,1.3495032,0.6666667,0.6795491,0.0132552,0.0405341,32.3920213,34.9503212
,4,0.04,0.6474268,1.2532254,1.3255211,0.6310680,0.6674728,0.0124868,0.0530208,25.3225444,32.5521083
,5,0.0500121,0.6380550,1.2759521,1.3155977,0.6425121,0.6624758,0.0127749,0.0657958,27.5952089,31.5597697
,6,0.1000242,0.6066923,1.2022834,1.2589406,0.6054159,0.6339458,0.0601287,0.1259245,20.2283443,25.8940570
,7,0.1500363,0.5853330,1.1734747,1.2304520,0.5909091,0.6196003,0.0586879,0.1846124,17.3474734,23.0451958
,8,0.2,0.5697029,1.1996024,1.2227452,0.6040658,0.6157195,0.0599366,0.2445490,19.9602439,22.2745173
,9,0.3000242,0.5424444,1.1072147,1.1842288,0.5575435,0.5963244,0.1107482,0.3552973,10.7214704,18.4228809
,10,0.4,0.5211215,1.0606734,1.1533474,0.5341074,0.5807739,0.1060417,0.4613390,6.0673378,15.3347421


In [42]:
#print test_metrics.__class__
#if str(test_metrics.__class__) == "<class 'h2o.model.metrics_base.H2OBinomialModelMetrics'>":
from h2o.model.metrics_base import H2OBinomialModelMetrics
if isinstance(test_metrics, H2OBinomialModelMetrics):
    print test_metrics.confusion_matrix(metrics="accuracy")
    print 'AUC:', test_metrics.auc()
    print 'Accuracy:', test_metrics.accuracy()

Confusion Matrix (Act/Pred) for max accuracy @ threshold = 0.497682978678: 


,DETRACTOR,NON DETRACTOR,Error,Rate
DETRACTOR,5605.0,4659.0,0.4539,(4659.0/10264.0)
NON DETRACTOR,4291.0,6120.0,0.4122,(4291.0/10411.0)
Total,9896.0,10779.0,0.4329,(8950.0/20675.0)



AUC: 0.592198721966
Accuracy: [[0.49768297867849487, 0.5671100362756953]]


### Oracle Non-Prepaid segment - Predictions

Make predictions

In [43]:
vf_pos.printSchema()

root
 |-- nif: string (nullable = true)
 |-- partitioned_month: string (nullable = true)
 |-- AC_prodhz: string (nullable = true)
 |-- AC_prodadsl: string (nullable = true)
 |-- AC_x_plan: string (nullable = true)
 |-- AC_promocion_vf: string (nullable = true)
 |-- AC_prodlpd: string (nullable = true)
 |-- AC_puntos: integer (nullable = true)
 |-- AC_sistema_operativo: string (nullable = true)
 |-- AC_promocion_tarifa: string (nullable = true)
 |-- AC_flag_huella_von: string (nullable = true)
 |-- AC_prodtivo: string (nullable = true)
 |-- AC_x_tipo_ident: string (nullable = true)
 |-- AC_part_status: string (nullable = true)
 |-- AC_pprecios_destino: string (nullable = true)
 |-- AC_x_sexo: string (nullable = true)
 |-- AC_plandatos: string (nullable = true)
 |-- AC_ppid_destino: string (nullable = true)
 |-- AC_cod_segfid: integer (nullable = true)
 |-- AC_modelo: string (nullable = true)
 |-- AC_tarifa_canje: string (nullable = true)
 |-- AC_x_nacionalidad: string (nullable = true)


In [44]:
h2o.ls()

,key
0,AutoML_20180410_163527
1,AutoML_20180410_164942
2,AutoML_Feedback_AutoML_20180410_163527
3,AutoML_Feedback_AutoML_20180410_164942
4,AutoML_Leaderboard_automl_py_7_sid_af3d
5,DRF_0_AutoML_20180410_164942
6,DRF_0_AutoML_20180410_164942_cv_1
7,DRF_0_AutoML_20180410_164942_cv_2
8,DRF_0_AutoML_20180410_164942_cv_3
9,DRF_0_AutoML_20180410_164942_cv_4


In [45]:
vf_pos.write.mode('overwrite').format('parquet').save('/tmp/bbergua/tmp/vf_pos_all-'+month)

In [46]:
# $ hdfs getconf -confKey fs.defaultFS
# hdfs://nameservice1
# $ hdfs getconf -namenodes
# vgddp350hr.dc.sedc.internal.vodafone.com vgddp351hr.dc.sedc.internal.vodafone.com
out = subprocess.check_output('hdfs getconf -namenodes', shell=True)
nodes = ['hdfs://'+n for n in out.strip().split(' ')]

In [47]:
path_vf_pos_all = [n+'/tmp/bbergua/tmp/vf_pos_all-'+month for n in nodes]
path_vf_pos_all[0]

'hdfs://vgddp350hr.dc.sedc.internal.vodafone.com/tmp/bbergua/tmp/vf_pos_all-201712'

In [48]:
vf_pos_all_df_h2o = h2o.import_file(path=path_vf_pos_all[0])
vf_pos_all_df_h2o.nrows
#subprocess.call('hdfs dfs -rm -r /tmp/bbergua/tmp/vf_pos_all-'+month, shell=True)

Parse progress: |█████████████████████████████████████████████████████████| 100%


3285656

In [51]:
if False:
    #h2o.remove_all()
    #vf_pos_all_pd = generate_pandas_data(vf_pos)

    # Copy to H2O

    #print balanced_pd.columns.values.tolist()
    #os.environ["PYTHONIOENCODING"] = "UTF-8"
    print 'Copying data to H2O'
    h2o.remove('vf_pos_all_df_h2o.hex')
    vf_pos_all_df_h2o = h2o.H2OFrame(vf_pos_all_pd, destination_frame='vf_pos_all_df_h2o.hex', header=1)
    vf_pos_all_df_h2o.head()

    #preds = aml.leader.predict()

In [54]:
#vf_pos_all_pd.dtypes
#print vf_pos_all_pd.columns.values.tolist()

In [55]:
h2o.ls()

,key
0,AutoML_20180410_163527
1,AutoML_20180410_164942
2,AutoML_Feedback_AutoML_20180410_163527
3,AutoML_Feedback_AutoML_20180410_164942
4,AutoML_Leaderboard_automl_py_7_sid_af3d
5,DRF_0_AutoML_20180410_164942
6,DRF_0_AutoML_20180410_164942_cv_1
7,DRF_0_AutoML_20180410_164942_cv_2
8,DRF_0_AutoML_20180410_164942_cv_3
9,DRF_0_AutoML_20180410_164942_cv_4


In [56]:
#vf_pos_all_df_h2o[''] = vf_pos_all_df_h2o[''].asnumeric()
#vf_pos_all_df_h2o[''] = vf_pos_all_df_h2o[''].asnumeric()
#vf_pos_all_df_h2o[''] = vf_pos_all_df_h2o[''].asnumeric()

In [57]:
vf_pos_all_df_h2o.describe()

Rows:3285656
Cols:282




,nif,partitioned_month,AC_prodhz,AC_prodadsl,AC_x_plan,AC_promocion_vf,AC_prodlpd,AC_puntos,AC_sistema_operativo,AC_promocion_tarifa,AC_flag_huella_von,AC_prodtivo,AC_x_tipo_ident,AC_part_status,AC_pprecios_destino,AC_x_sexo,AC_plandatos,AC_ppid_destino,AC_cod_segfid,AC_modelo,AC_tarifa_canje,AC_x_nacionalidad,AC_flag_cobertura_adsl,AC_codigo_postal,AC_x_dia_emision,AC_x_subtipo,AC_prodvfbox,AC_prodftth,AC_prodadslenprov,AC_x_tipo_cliente,AC_sum_flagvfbox,AC_sum_flaghz,AC_sum_flagvoz,AC_sum_flagtivo,AC_sum_flaglpd,AC_sum_flagfutbol,AC_sum_flagftth,AC_sum_flagadsl,AC_sum_arpu,AC_sum_cantidad_pendiente,AC_sum_cuotas_pendientes,AC_is_mobile_only_new,AC_is_mobile_only,AC_is_convergent_new,AC_is_convergent,AC_max_cobertura_4g,AC_max_flag_huella_vf,AC_max_flag_ebilling,AC_max_lortad,AC_max_flag_financia,AC_max_vfsmartphone,AC_max_flag_nh_real,AC_max_seg_cliente,AC_max_flag_4g_nodos,AC_max_flag_existe_fbb_hogar,AC_max_flag_siebel,AC_max_flag_4g,AC_max_flag_huella_movistar,AC_max_terminal_4g,AC_max_flag_desc_conv,AC_max_flag_financia_simo,AC_max_flag_4g_aperturas,AC_max_flag_huella_jazztel,AC_max_terminalmms,AC_max_num_pospago,AC_max_flag_huella_neba,AC_max_flag_nh_prevista,AC_max_deuda,AC_max_cobertura_4g_plus,AC_max_flag_huella_ono,AC_max_num_prepago,AC_max_flag_cuenta_superintegral,AC_max_flag_huella_euskaltel,AC_max_num_total,AC_min_days_since_install_date,AC_min_days_since_fecha_fin_cp_tarifa,AC_min_days_since_x_fecha_nacimiento,AC_min_days_since_x_fecha_activacion,AC_min_days_since_fecha_fin_cp_vf,AC_min_days_since_fecha_alta_ss_ftth,AC_min_days_since_fecha_alta_ss_adsl,AC_min_days_since_fecha_alta_tivo,AC_min_days_since_fecha_alta_ss_hz,AC_min_days_since_fecha_alta_ss_adslenprov,AC_min_days_since_fecha_alta_vfbox,AC_min_days_since_fecha_ultima_financiacion,AC_min_days_since_fecha_alta_futbol,AC_min_days_since_fecha_alta_servicio,AC_min_days_since_fecha_transferencia,AC_min_days_since_fecha_antiguedad_cliente,AC_min_days_since_fecha_canje,AC_min_days_since_x_fecha_creacion_cuenta,AC_min_days_since_last_upd,AC_min_days_since_fecha_cambio,AC_min_days_since_x_fecha_recepcion,AC_min_days_since_x_fecha_ini_prov,AC_min_days_since_x_fecha_creacion_servicio,AC_min_x_dia_emision,AC_min_years_since_install_date,AC_min_years_since_fecha_fin_cp_tarifa,AC_min_years_since_x_fecha_nacimiento,AC_min_years_since_x_fecha_activacion,AC_min_years_since_fecha_fin_cp_vf,AC_min_years_since_fecha_alta_ss_ftth,AC_min_years_since_fecha_alta_ss_adsl,AC_min_years_since_fecha_alta_tivo,AC_min_years_since_fecha_alta_ss_hz,AC_min_years_since_fecha_alta_ss_adslenprov,AC_min_years_since_fecha_alta_vfbox,AC_min_years_since_fecha_ultima_financiacion,AC_min_years_since_fecha_alta_futbol,AC_min_years_since_fecha_alta_servicio,AC_min_years_since_fecha_transferencia,AC_min_years_since_fecha_antiguedad_cliente,AC_min_years_since_fecha_canje,AC_min_years_since_x_fecha_creacion_cuenta,AC_min_years_since_last_upd,AC_min_years_since_fecha_cambio,AC_min_years_since_x_fecha_recepcion,AC_min_years_since_x_fecha_ini_prov,AC_min_years_since_x_fecha_creacion_servicio,AC_avg_days_since_install_date,AC_avg_days_since_fecha_fin_cp_tarifa,AC_avg_days_since_x_fecha_nacimiento,AC_avg_days_since_x_fecha_activacion,AC_avg_days_since_fecha_fin_cp_vf,AC_avg_days_since_fecha_alta_ss_ftth,AC_avg_days_since_fecha_alta_ss_adsl,AC_avg_days_since_fecha_alta_tivo,AC_avg_days_since_fecha_alta_ss_hz,AC_avg_days_since_fecha_alta_ss_adslenprov,AC_avg_days_since_fecha_alta_vfbox,AC_avg_days_since_fecha_ultima_financiacion,AC_avg_days_since_fecha_alta_futbol,AC_avg_days_since_fecha_alta_servicio,AC_avg_days_since_fecha_transferencia,AC_avg_days_since_fecha_antiguedad_cliente,AC_avg_days_since_fecha_canje,AC_avg_days_since_x_fecha_creacion_cuenta,AC_avg_days_since_last_upd,AC_avg_days_since_fecha_cambio,AC_avg_days_since_x_fecha_recepcion,AC_avg_days_since_x_fecha_ini_prov,AC_avg_days_since_x_fecha_creacion_servicio,AC_avg_x_dia_emision,AC_avg_years_since_install_date,AC_avg_years_since_fecha_fin_cp_t

In [58]:
vf_pos_all_df_h2o_predict = vf_pos_all_df_h2o.drop(['TNPS01', 'TNPS2DET', 'TNPS2PRO', 'TNPS', 'TNPS4'])
vf_pos_all_preds = aml.predict(vf_pos_all_df_h2o_predict)

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/opt/cloudera/parcels/Anaconda/lib/python2.7/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'AC_prodadsl' has levels not trained on: [AAD03, AAD20, AAI03, AAI10, ADO20, AIOV3, DNM21, DNM22, DNM23, DNM24, DNM34, DNM42, DNM43, DNM44, DTT40, DTT42, DTU42, DTU54, DTU55, DTU64, DTX13, DTX65, DTX69]
  warnings.warn(w)
/opt/cloudera/parcels/Anaconda/lib/python2.7/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'AC_promocion_vf' has levels not trained on: [ADSINSOLO, AYD24400, CAPMC24PSG, CAPSINTER, CAPTOPTBOT, CAPVDIGIBO, CAPVF00SIM, CAPVF24400, CAPVF24ANY, CAPVF24AV, CAPVF24BA, CAPVF24MI2, CAPVF24PNY, CAPVFNYHIG, CAPVFNYLOW, CAPVFPAPSS, CAPVFVSMVS, CAPVFVSORA, COL24100, COL24400, COLSIMINT, COLT24400, COLVF24PR2, COLVFSIMO, CPDATOS18, CPP12150, CPP19000, CPP21000, CPP24060, CPP24073, CPP24096, CPP24097, CPP24098, CPP24100, CPP24120, CPP24121, CPP24169, CPP24289, CPP24300, CPP5006, CPP5012, CRUVF24ELI, D060610, D121220, FAMTAB_ES, FAMTAB_PR, 

In [59]:
#vf_pos_all_preds.head()

Append NIF and actual labels to predictions

In [60]:
vf_pos_all_preds_id = vf_pos_all_preds.cbind(vf_pos_all_df_h2o[['nif', label_preds, label_model]])
vf_pos_all_preds_id

predict,DETRACTOR,NON DETRACTOR,nif,TNPS,TNPS2DET
NON DETRACTOR,0.49559,0.50441,00018573Q,,
NON DETRACTOR,0.439892,0.560108,00027620Y,,
NON DETRACTOR,0.479954,0.520046,00028706G,,
NON DETRACTOR,0.514411,0.485589,00045283V,PROMOTER,NON DETRACTOR
NON DETRACTOR,0.498984,0.501016,00049003C,,
NON DETRACTOR,0.468719,0.531281,00049353D,,
NON DETRACTOR,0.579059,0.420941,00066859U,,
NON DETRACTOR,0.427448,0.572552,00080906R,,
NON DETRACTOR,0.604798,0.395202,00088614V,,
NON DETRACTOR,0.490356,0.509644,00100487X,,


Count number of rows and proportion of predictions

In [61]:
vf_pos_all_preds_nrows = vf_pos_all_preds_id.nrows
vf_pos_all_preds_groupby = vf_pos_all_preds_id.group_by(by='predict').count().get_frame()
proportion = 100*vf_pos_all_preds_groupby['nrow']/vf_pos_all_preds_nrows
proportion.set_names(['proportion'])
vf_pos_all_preds_groupby.cbind(proportion)

predict,nrow,proportion
DETRACTOR,11418,0.347511
NON DETRACTOR,3.27424e+06,99.6525


As proportions of predictions are highly unbalanced, let's create a new prediction column (**predict2**) using a threshold on scoring column

In [62]:
th_max_acc = test_metrics.accuracy()[0][0]
vf_pos_all_preds_id[vf_pos_all_preds_id["NON DETRACTOR"] >= th_max_acc, "predict2"] = "NON DETRACTOR"
vf_pos_all_preds_id[vf_pos_all_preds_id["NON DETRACTOR"] < th_max_acc, "predict2"] = "DETRACTOR"
vf_pos_all_preds_id['predict2'] = vf_pos_all_preds_id['predict2'].asfactor()
vf_pos_all_preds_id

predict,DETRACTOR,NON DETRACTOR,nif,TNPS,TNPS2DET,predict2
NON DETRACTOR,0.49559,0.50441,00018573Q,,,NON DETRACTOR
NON DETRACTOR,0.439892,0.560108,00027620Y,,,NON DETRACTOR
NON DETRACTOR,0.479954,0.520046,00028706G,,,NON DETRACTOR
NON DETRACTOR,0.514411,0.485589,00045283V,PROMOTER,NON DETRACTOR,DETRACTOR
NON DETRACTOR,0.498984,0.501016,00049003C,,,NON DETRACTOR
NON DETRACTOR,0.468719,0.531281,00049353D,,,NON DETRACTOR
NON DETRACTOR,0.579059,0.420941,00066859U,,,DETRACTOR
NON DETRACTOR,0.427448,0.572552,00080906R,,,NON DETRACTOR
NON DETRACTOR,0.604798,0.395202,00088614V,,,DETRACTOR
NON DETRACTOR,0.490356,0.509644,00100487X,,,NON DETRACTOR


Compare real label with this **predict2**  column newly created

In [63]:
vf_pos_all_preds_nrows = vf_pos_all_preds_id.nrows
vf_pos_all_preds_groupby = vf_pos_all_preds_id.group_by(by=['TNPS2DET', 'predict2']).count().get_frame()
proportion = 100*vf_pos_all_preds_groupby['nrow']/vf_pos_all_preds_nrows
proportion.set_names(['proportion'])
vf_pos_all_preds_groupby.cbind(proportion)

TNPS2DET,predict2,nrow,proportion
,DETRACTOR,1.5524e+06,47.2479
,NON DETRACTOR,1.44852e+06,44.0861
DETRACTOR,DETRACTOR,51657,1.5722
DETRACTOR,NON DETRACTOR,17518,0.533166
NON DETRACTOR,DETRACTOR,76410,2.32556
NON DETRACTOR,NON DETRACTOR,139152,4.23514


Count number of rows and proportion of predictions using the new **predict2** column

In [64]:
vf_pos_all_preds_nrows = vf_pos_all_preds_id.nrows
vf_pos_all_preds_groupby = vf_pos_all_preds_id.group_by(by='predict2').count().get_frame()
proportion = 100*vf_pos_all_preds_groupby['nrow']/vf_pos_all_preds_nrows
proportion.set_names(['proportion'])
vf_pos_all_preds_groupby.cbind(proportion)

predict2,nrow,proportion
DETRACTOR,1.68047e+06,51.1456
NON DETRACTOR,1.60519e+06,48.8544


As what we really need is to provide a prediction of the type ['PROMOTER', 'NEUTRAL', 'DETRACTOR'], we calculate cut points in predictions to preserve original proportions

In [65]:
print 'nrows:', vf_pos_all_preds_nrows
#print 'proportions', vf_pos_proportions


if 'HARD DETRACTOR' in vf_pos_proportions.keys() and 'SOFT DETRACTOR' in vf_pos_proportions.keys():
    vf_pos_proportions['DETRACTOR'] = vf_pos_proportions['HARD DETRACTOR'] + vf_pos_proportions['SOFT DETRACTOR']
    del vf_pos_proportions['HARD DETRACTOR']
    del vf_pos_proportions['SOFT DETRACTOR']

vf_pos_counts = {}
for k in vf_pos_proportions.keys():
    vf_pos_counts[k] = int(vf_pos_all_preds_nrows*vf_pos_proportions[k]/100)
diff = vf_pos_all_preds_nrows - sum(vf_pos_counts.values())
min_key = min(vf_pos_counts, key=vf_pos_counts.get) # Get the key with minimum value
vf_pos_counts[min_key] = vf_pos_counts[min_key] + diff # Assign rest to key with minimum num rows
print 'counts:', vf_pos_counts

ini = 0
end = 0
vf_pos_ranges = {}
for k in ['DETRACTOR', 'NEUTRAL', 'PROMOTER']: # It is necessary to force the order
    if end > 0:
        ini = end + 1
    end = ini + vf_pos_counts[k] - 1
    vf_pos_ranges[k] = [ini, end]
    #print k, ini, end
print 'ranges:', vf_pos_ranges

nrows: 3285656
counts: {u'NEUTRAL': 701623, u'PROMOTER': 1785805, 'DETRACTOR': 798228}
ranges: {'NEUTRAL': [798228, 1499850], 'PROMOTER': [1499851, 3285655], 'DETRACTOR': [0, 798227]}


Using the cut points calculated above, create a new **predict3** column with final predictions

In [66]:
for k in vf_pos_ranges.keys():
    r = range(vf_pos_ranges[k][0], vf_pos_ranges[k][1]+1, 1)
    print k, r[0], r[-1]
    vf_pos_all_preds_id[r, 'predict3'] = k
    #tmp = vf_pos_all_preds_id[r, :]
    #print tmp.head()
vf_pos_all_preds_id['predict3'] = vf_pos_all_preds_id['predict3'].asfactor()
vf_pos_all_preds_id

NEUTRAL 798228 1499850
PROMOTER 1499851 3285655
DETRACTOR 0 798227


predict,DETRACTOR,NON DETRACTOR,nif,TNPS,TNPS2DET,predict2,predict3
NON DETRACTOR,0.49559,0.50441,00018573Q,,,NON DETRACTOR,DETRACTOR
NON DETRACTOR,0.439892,0.560108,00027620Y,,,NON DETRACTOR,DETRACTOR
NON DETRACTOR,0.479954,0.520046,00028706G,,,NON DETRACTOR,DETRACTOR
NON DETRACTOR,0.514411,0.485589,00045283V,PROMOTER,NON DETRACTOR,DETRACTOR,DETRACTOR
NON DETRACTOR,0.498984,0.501016,00049003C,,,NON DETRACTOR,DETRACTOR
NON DETRACTOR,0.468719,0.531281,00049353D,,,NON DETRACTOR,DETRACTOR
NON DETRACTOR,0.579059,0.420941,00066859U,,,DETRACTOR,DETRACTOR
NON DETRACTOR,0.427448,0.572552,00080906R,,,NON DETRACTOR,DETRACTOR
NON DETRACTOR,0.604798,0.395202,00088614V,,,DETRACTOR,DETRACTOR
NON DETRACTOR,0.490356,0.509644,00100487X,,,NON DETRACTOR,DETRACTOR


Count number of rows and proportion of predictions using the final **predict3** column, that must match original proportions

In [67]:
vf_pos_all_preds_nrows = vf_pos_all_preds_id.nrows
vf_pos_all_preds_groupby = vf_pos_all_preds_id.group_by(by='predict3').count().get_frame()
proportion = 100*vf_pos_all_preds_groupby['nrow']/vf_pos_all_preds_nrows
proportion.set_names(['proportion'])
vf_pos_all_preds_groupby.cbind(proportion)

predict3,nrow,proportion
DETRACTOR,798228,24.2943
NEUTRAL,701623,21.3541
PROMOTER,1.78580e+06,54.3516


Convert predictions to Pandas, to bring predictions back to Spark again

In [68]:
start = time.time()
print 'Converting predictions to Pandas'
vf_pos_all_preds_id_pd = vf_pos_all_preds_id[['nif', label_preds, 'predict3']].as_data_frame()
#vf_pos_all_preds_id_pd
end = time.time()
print 'Process took:', "{0:.2f}".format((end - start)/60), 'minutes'

Converting predictions to Pandas
Process took: 0.14 minutes


Create a Spark DataFrame from the Pandas object created above

In [69]:
#Create PySpark DataFrame Schema
pd_schema = StructType([StructField('nif',        StringType(),True),
                        StructField('label_preds',StringType(),True),
                        StructField('predict',    StringType(),True)])

#Create Spark DataFrame from Pandas
df_vf_pos_preds = sqlContext.createDataFrame(vf_pos_all_preds_id_pd, pd_schema)
#Important to order columns in the same order as the target database
df_vf_pos_preds = df_vf_pos_preds.select('nif', 'label_preds', 'predict')

Count number of rows of actual label and final predictions

In [70]:
print df_vf_pos_preds.count()
df_vf_pos_preds.groupby('label_preds').count().show()
#print df_vf_pos_preds.filter('label_preds == "NaN"').count()
df_vf_pos_preds.groupby('predict').count().show()

3285656
+-----------+-------+
|label_preds|  count|
+-----------+-------+
|    NEUTRAL|  60803|
|   PROMOTER| 154759|
|        NaN|3000919|
|  DETRACTOR|  69175|
+-----------+-------+

+---------+-------+
|  predict|  count|
+---------+-------+
|  NEUTRAL| 701623|
| PROMOTER|1785805|
|DETRACTOR| 798228|
+---------+-------+



For those clients that we really have actual label, use actual label. Otherwise, use prediction.

In [71]:
vf_pos_preds = df_vf_pos_preds.withColumn('predict', 
                                          when(df_vf_pos_preds['label_preds'] != "NaN",
                                               df_vf_pos_preds['label_preds'])\
                                         .otherwise(df_vf_pos_preds['predict']))\
                              .drop('label_preds')
#print vf_pos_preds.count()
#vf_pos_preds.groupby('predict').count().show()

vf_pos_preds_count = vf_pos_preds.count()
print vf_pos_preds_count
vf_pos_preds_groupby = vf_pos_preds.groupby('predict').count()
vf_pos_preds_proportions = vf_pos_preds_groupby.withColumn('proportion', 100*vf_pos_preds_groupby['count']/vf_pos_preds_count).cache()
vf_pos_preds_proportions.withColumn('proportion', bround(vf_pos_preds_proportions['proportion'], 2)).show()

3285656
+---------+-------+----------+
|  predict|  count|proportion|
+---------+-------+----------+
|  NEUTRAL| 701574|     21.35|
| PROMOTER|1785774|     54.35|
|DETRACTOR| 798308|      24.3|
+---------+-------+----------+



Write predictions to HDFS

In [72]:
vf_pos_preds.write.mode('overwrite').format('parquet').save('/tmp/bbergua/tmp/vf_pos_preds-'+month)

## Write predictions to HDFS

In [99]:
if not 'vf_pre_preds' in globals():
    print 'Reading Vodafone Prepaid predictions from HDFS for month =', month
    vf_pre_preds = spark.read.parquet('/tmp/bbergua/tmp/vf_pre_preds-'+month)

In [76]:
vf_pre_preds.printSchema()

root
 |-- nif: string (nullable = true)
 |-- predict: string (nullable = true)



In [77]:
print vf_pre_preds.select('nif').count()
print vf_pre_preds.select('nif').distinct().count()

2267021
2267021


In [98]:
if not 'vf_pos_preds' in globals():
    print 'Reading Vodafone Postpaid predictions from HDFS for month =', month
    vf_pos_preds = spark.read.parquet('/tmp/bbergua/tmp/vf_pos_preds-'+month)

In [78]:
vf_pos_preds.printSchema()

root
 |-- nif: string (nullable = true)
 |-- predict: string (nullable = true)



In [79]:
print vf_pos_preds.select('nif').count()
print vf_pos_preds.select('nif').distinct().count()

3285656
3285656


Write predictions to HDFS

In [95]:
# For those NIFs shared between Prepaid and Postpaid, drop them from Prepaid DataFrame (and thus, take them from Postpaid)
shared = vf_pre_preds.join(vf_pos_preds.withColumnRenamed('predict', 'predict_pos'), 'nif', 'left_outer')
vf_pre_preds = shared.filter('predict_pos is NULL').drop('predict_pos')

oracle_preds = vf_pre_preds.select('nif', 'predict').union(vf_pos_preds.select('nif', 'predict'))

In [96]:
print oracle_preds.select('nif').count()
print oracle_preds.select('nif').distinct().count()

5281435
5281435


In [97]:
oracle_preds.write.mode('overwrite').format('parquet').save('/tmp/bbergua/tmp/oracle_preds-'+month)